In [1]:
!pip install -e .

Obtaining file:///Users/matthewdeane/Documents/Data%20Science/python/_projects/py-tidymodels/_md
ERROR: file:///Users/matthewdeane/Documents/Data%20Science/python/_projects/py-tidymodels/_md does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# py-tidymodels imports
from py_workflows import workflow
from py_parsnip import linear_reg
from py_rsample import vfold_cv, initial_split, training, testing
from py_yardstick import metric_set, rmse, mae, r_squared, smape
from py_workflowsets import WorkflowSet

from py_recipes import recipe
from py_recipes.selectors import all_numeric, all_nominal, starts_with, ends_with

from py_visualize import plot_forecast, plot_residuals

# Set random seed for reproducibility
np.random.seed(42)

# Import Data

In [3]:
# import raw data
raw_data = pd.read_csv('__data/preem.csv')

In [4]:
df = raw_data

# set date as index
#df = raw_data.set_index('date')

# convert date to datetime
df['date'] = pd.to_datetime(df['date'])
display(df)

,date,mean_med_diesel_crack_input1_trade_month_lag2,mean_nwe_hsfo_crack_trade_month_lag1,mean_nwe_lsfo_crack_trade_month,mean_nwe_ulsfo_crack_trade_month_lag3,mean_sing_gasoline_vs_vlsfo_trade_month,mean_sing_vlsfo_crack_trade_month_lag3,new_sweet_sr_margin,target,totaltar
0,2020-04-01,-56.18,-11.43,-3.07,28.58,-16.09,26.91,0.47,137.65,0.00
1,2020-05-01,-42.36,-9.17,3.89,22.64,-8.75,20.36,0.57,113.53,0.00
2,2020-06-01,-30.21,-8.58,-3.44,13.43,-4.28,11.32,6.61,43.31,0.00
3,2020-07-01,-28.86,-6.86,-2.71,10.74,-3.20,8.53,-1.55,79.77,0.00
4,2020-08-01,-38.80,-5.86,-2.63,4.58,-5.35,6.26,-2.70,48.27,0.00
5,2020-09-01,-41.76,-5.62,-2.08,7.69,-3.26,6.15,-2.10,98.01,0.00
6,2020-10-01,-43.99,-5.10,0.16,8.09,-4.31,5.80,-3.34,49.77,0.00
7,2020-11-01,-42.94,-5.43,1.80,7.86,-7.40,6.99,-2.53,35.48,0.00
8,2020-12-01,-41.55,-3.39,0.94,6.68,-7.65,6.22,0.00,63.98,0.00
9,2021-01-01,-41.18,-6.03,0.63,8.67,-7.67,8.22,-0.05,109.76,0.00


# py-rsample

In [5]:
from py_rsample import initial_time_split, time_series_cv

In [6]:
# Create train/test split
split = initial_split(df, prop=0.75, seed=123)
train_data = training(split)
test_data = testing(split)

print(f"Training set: {train_data.shape[0]} rows")
print(f"Test set: {test_data.shape[0]} rows")

Training set: 42 rows
Test set: 15 rows


In [7]:
# Example: Train on 2020, test on first half of 2021
split_absolute = initial_time_split(
    df,
    date_column="date",
    train_start="2022-01-01",
    train_end="2023-12-01",
    test_start="2024-01-01",
    test_end="2024-06-01"
)

train_abs = split_absolute.training()
test_abs = split_absolute.testing()

print(f"Training: {len(train_abs)} days from {train_abs['date'].min()} to {train_abs['date'].max()}")
print(f"Testing:  {len(test_abs)} days from {test_abs['date'].min()} to {test_abs['date'].max()}")

# bug - thinks its days not months (monthly data)

Training: 24 days from 2022-01-01 00:00:00 to 2023-12-01 00:00:00
Testing:  6 days from 2024-01-01 00:00:00 to 2024-06-01 00:00:00


In [8]:
# Example 1: Train on first 18 months, test on last 6 months
split_relative1 = initial_time_split(
    df,
    date_column="date",
    train_start="start",
    train_end="start + 18 months",
    test_start="end - 6 months",
    test_end="end"
)

train_rel1 = split_relative1.training()
test_rel1 = split_relative1.testing()

print("Example 1: First 18 months vs last 6 months")
print(f"Training: {len(train_rel1)} days from {train_rel1['date'].min()} to {train_rel1['date'].max()}")
print(f"Testing:  {len(test_rel1)} days from {test_rel1['date'].min()} to {test_rel1['date'].max()}")
print()

# bug - thinks its days not months (monthly data)

Example 1: First 18 months vs last 6 months
Training: 18 days from 2020-04-01 00:00:00 to 2021-09-01 00:00:00
Testing:  6 days from 2024-07-01 00:00:00 to 2024-12-01 00:00:00



In [9]:
# Train on specific year (2020), test on last 3 months of available data
split_mixed = initial_time_split(
    df,
    date_column="date",
    train_start="start",
    train_end="2023-12-01",
    test_start="end - 12 months",
    test_end="end"
)

train_mix = split_mixed.training()
test_mix = split_mixed.testing()

print("Mixed: Train on 2020, test on last 90 days of data")
print(f"Training: {len(train_mix)} days from {train_mix['date'].min()} to {train_mix['date'].max()}")
print(f"Testing:  {len(test_mix)} days from {test_mix['date'].min()} to {test_mix['date'].max()}")

# bug - thinks its days not months (monthly data)

Mixed: Train on 2020, test on last 90 days of data
Training: 45 days from 2020-04-01 00:00:00 to 2023-12-01 00:00:00
Testing:  12 days from 2024-01-01 00:00:00 to 2024-12-01 00:00:00


# 1. Single Model Fitting

In [10]:
df.columns

Index(['date', 'mean_med_diesel_crack_input1_trade_month_lag2',
       'mean_nwe_hsfo_crack_trade_month_lag1',
       'mean_nwe_lsfo_crack_trade_month',
       'mean_nwe_ulsfo_crack_trade_month_lag3',
       'mean_sing_gasoline_vs_vlsfo_trade_month',
       'mean_sing_vlsfo_crack_trade_month_lag3', 'new_sweet_sr_margin',
       'target', 'totaltar'],
      dtype='object')

In [11]:
FORMULA_STR = "target ~ totaltar +mean_med_diesel_crack_input1_trade_month_lag2 + mean_nwe_hsfo_crack_trade_month_lag1"
FORMULA_STR = "target ~ ."
# FORMULA_STR = "target ~ totaltar +mean_med_diesel_crack_input1_trade_month_lag2 + mean_nwe_hsfo_crack_trade_month_lag1 + mean_nwe_lsfo_crack_trade_month + mean_nwe_ulsfo_crack_trade_month_lag3 + mean_sing_gasoline_vs_vlsfo_trade_month + mean_sing_vlsfo_crack_trade_month_lag3 + new_sweet_sr_margin"

## 1.1 - Prophet

In [12]:
# SINGLE MODEL fitting
from py_visualize import plot_forecast
from py_parsnip import prophet_reg, arima_reg, rand_forest, linear_reg

# Create Prophet specification
# Prophet will automatically detect weekly seasonality in our data
spec_prophet = prophet_reg(
    n_changepoints=25,  # Number of potential trend changes
    changepoint_prior_scale=0.05,  # Flexibility of trend
    seasonality_prior_scale=10.0   # Flexibility of seasonality
)

print(spec_prophet)

# # Fit Prophet - FAILING
fit_prophet = spec_prophet.fit(train_data, FORMULA_STR)
print("Prophet model fitted!")

# Evaluate and extract outputs
fit_prophet = fit_prophet.evaluate(test_data)
outputs_prophet, coefs_prophet, stats_prophet = fit_prophet.extract_outputs()

display(outputs_prophet)
display(coefs_prophet)
display(stats_prophet)

# Create interactive forecast plot
fig = plot_forecast(fit_prophet, title="Sales Forecast")
fig.show()

fig = plot_residuals(fit_prophet, title="Sales Forecast")
fig.show()

ModelSpec(model_type='prophet_reg', engine='prophet', mode='regression', args={'growth': 'linear', 'changepoint_prior_scale': 0.05, 'seasonality_prior_scale': 10.0, 'seasonality_mode': 'additive', 'n_changepoints': 25, 'changepoint_range': 0.8, 'yearly_seasonality': 'auto', 'weekly_seasonality': 'auto', 'daily_seasonality': 'auto'}, date_col=None)


18:06:07 - cmdstanpy - INFO - Chain [1] start processing
18:06:07 - cmdstanpy - INFO - Chain [1] done processing


Prophet model fitted!


,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,115.944878,137.65,21.705122,train,prophet_reg,,global
1,2020-05-01,113.53,118.598941,113.53,-5.068941,train,prophet_reg,,global
2,2020-06-01,43.31,68.304292,43.31,-24.994292,train,prophet_reg,,global
3,2020-07-01,79.77,72.219618,79.77,7.550382,train,prophet_reg,,global
4,2020-08-01,48.27,44.355810,48.27,3.914190,train,prophet_reg,,global
5,2020-09-01,98.01,110.773505,98.01,-12.763505,train,prophet_reg,,global
6,2020-10-01,49.77,70.704920,49.77,-20.934920,train,prophet_reg,,global
7,2020-11-01,35.48,38.467477,35.48,-2.987477,train,prophet_reg,,global
8,2020-12-01,63.98,88.571625,63.98,-24.591625,train,prophet_reg,,global
9,2021-01-01,109.76,116.650937,109.76,-6.890937,train,prophet_reg,,global


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,growth,linear,NaN,NaN,NaN,NaN,NaN,NaN,prophet_reg,,global
1,changepoint_prior_scale,0.05,NaN,NaN,NaN,NaN,NaN,NaN,prophet_reg,,global
2,seasonality_prior_scale,10.0,NaN,NaN,NaN,NaN,NaN,NaN,prophet_reg,,global
3,seasonality_mode,additive,NaN,NaN,NaN,NaN,NaN,NaN,prophet_reg,,global
4,n_changepoints,25,NaN,NaN,NaN,NaN,NaN,NaN,prophet_reg,,global


,metric,value,split,model,model_group_name,group
0,rmse,17.969607,train,prophet_reg,,global
1,mae,13.879322,train,prophet_reg,,global
2,mape,15.11841,train,prophet_reg,,global
3,smape,14.423303,train,prophet_reg,,global
4,r_squared,0.740229,train,prophet_reg,,global
5,mda,68.292683,train,prophet_reg,,global
6,rmse,79.938047,test,prophet_reg,,global
7,mae,71.511399,test,prophet_reg,,global
8,mape,57.202044,test,prophet_reg,,global
9,smape,85.011131,test,prophet_reg,,global


## 1.2 - ARIMA

In [13]:
# Create ARIMA specification
spec_arima = arima_reg(
    seasonal_period=7,  # Weekly seasonality
    non_seasonal_ar=1,
    non_seasonal_differences=1,
    non_seasonal_ma=1,
    seasonal_ar=1,
    seasonal_differences=0,
    seasonal_ma=1
)

print(spec_arima)

# Fit ARIMA
fit_arima = spec_arima.fit(train_data, FORMULA_STR)
print("ARIMA model fitted!")

# Predict on test data
pred_arima = fit_arima.predict(test_data)

# Evaluate and extract outputs
fit_arima = fit_arima.evaluate(test_data)
outputs_arima, coefs_arima, stats_arima = fit_arima.extract_outputs()

display(outputs_arima)
display(coefs_arima)
display(stats_arima)

# Create interactive forecast plot
fig = plot_forecast(fit_arima, title="Sales Forecast", prediction_intervals=True)
fig.show()

ModelSpec(model_type='arima_reg', engine='statsmodels', mode='regression', args={'seasonal_period': 7, 'non_seasonal_ar': 1, 'non_seasonal_differences': 1, 'non_seasonal_ma': 1, 'seasonal_ar': 1, 'seasonal_differences': 0, 'seasonal_ma': 1}, date_col=None)
ARIMA model fitted!


/Users/matthewdeane/Documents/Data Science/python/_projects/py-tidymodels/py-tidymodels2/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

/Users/matthewdeane/Documents/Data Science/python/_projects/py-tidymodels/py-tidymodels2/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.



,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,-122.922122,137.65,260.572122,train,arima_reg,,global
1,2020-05-01,113.53,184.075269,113.53,-70.545269,train,arima_reg,,global
2,2020-06-01,43.31,162.771180,43.31,-119.461180,train,arima_reg,,global
3,2020-07-01,79.77,28.032101,79.77,51.737899,train,arima_reg,,global
4,2020-08-01,48.27,80.435608,48.27,-32.165608,train,arima_reg,,global
5,2020-09-01,98.01,52.977943,98.01,45.032057,train,arima_reg,,global
6,2020-10-01,49.77,96.384451,49.77,-46.614451,train,arima_reg,,global
7,2020-11-01,35.48,31.233576,35.48,4.246424,train,arima_reg,,global
8,2020-12-01,63.98,45.514707,63.98,18.465293,train,arima_reg,,global
9,2021-01-01,109.76,63.724872,109.76,46.035128,train,arima_reg,,global


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,mean_med_diesel_crack_input1_trade_month_lag2,0.091674,0.081754,1.121346,0.262141,-0.068560,0.251909,NaN,arima_reg,,global
1,mean_nwe_hsfo_crack_trade_month_lag1,0.199768,0.491865,0.406145,0.684636,-0.764269,1.163805,NaN,arima_reg,,global
2,mean_nwe_lsfo_crack_trade_month,-0.397631,2.278989,-0.174477,0.861491,-4.864368,4.069106,NaN,arima_reg,,global
3,mean_nwe_ulsfo_crack_trade_month_lag3,0.221236,2.644588,0.083656,0.933330,-4.962060,5.404532,NaN,arima_reg,,global
4,mean_sing_gasoline_vs_vlsfo_trade_month,2.441624,1.831307,1.333268,0.182444,-1.147672,6.030920,NaN,arima_reg,,global
5,mean_sing_vlsfo_crack_trade_month_lag3,-3.147952,3.853830,-0.816837,0.414021,-10.701321,4.405416,NaN,arima_reg,,global
6,new_sweet_sr_margin,2.053143,1.955285,1.050048,0.293696,-1.779145,5.885431,NaN,arima_reg,,global
7,totaltar,0.086354,0.426915,0.202274,0.839702,-0.750384,0.923092,NaN,arima_reg,,global
8,ar.L1,0.077081,0.328844,0.234400,0.814674,-0.567441,0.721603,NaN,arima_reg,,global
9,ma.L1,-0.849687,0.292210,-2.907797,0.003640,-1.422407,-0.276966,NaN,arima_reg,,global


,metric,value,split,model,model_group_name,group
0,rmse,52.315572,train,arima_reg,,global
1,mae,31.198339,train,arima_reg,,global
2,mape,35.956461,train,arima_reg,,global
3,smape,31.606354,train,arima_reg,,global
4,r_squared,-1.201785,train,arima_reg,,global
5,mda,51.219512,train,arima_reg,,global
6,rmse,50.162797,test,arima_reg,,global
7,mae,44.260276,test,arima_reg,,global
8,mape,33.918292,test,arima_reg,,global
9,smape,42.478662,test,arima_reg,,global


## 1.3 - OLS - Statsmodels

In [14]:
from py_parsnip import linear_reg

# Create specification with statsmodels engine
spec_sm = linear_reg().set_engine("statsmodels")

# Fit model
fit_sm = spec_sm.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_sm = fit_sm.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs from both models
outputs_sm, coefs_sm, stats_sm = fit_sm.extract_outputs()

display(outputs_sm)
display(coefs_sm)
display(stats_sm)

fig = plot_forecast(fit_sm, title="Sales Forecast")
fig.show()

Model fitted successfully!


,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,100.399274,137.65,37.250726,train,linear_reg,,global
1,2020-05-01,113.53,98.415778,113.53,15.114222,train,linear_reg,,global
2,2020-06-01,43.31,102.319372,43.31,-59.009372,train,linear_reg,,global
3,2020-07-01,79.77,95.580130,79.77,-15.810130,train,linear_reg,,global
4,2020-08-01,48.27,89.088577,48.27,-40.818577,train,linear_reg,,global
5,2020-09-01,98.01,92.715549,98.01,5.294451,train,linear_reg,,global
6,2020-10-01,49.77,89.049033,49.77,-39.279033,train,linear_reg,,global
7,2020-11-01,35.48,85.125478,35.48,-49.645478,train,linear_reg,,global
8,2020-12-01,63.98,85.064946,63.98,-21.084946,train,linear_reg,,global
9,2021-01-01,109.76,88.002123,109.76,21.757877,train,linear_reg,,global


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,Intercept,86.200233,17.986252,4.792562,0.000034,49.606927,122.793538,NaN,linear_reg,,global
1,mean_med_diesel_crack_input1_trade_month_lag2,-0.062735,0.112253,-0.558873,0.580022,-0.291116,0.165645,1.240473,linear_reg,,global
2,mean_nwe_hsfo_crack_trade_month_lag1,-0.321839,0.496315,-0.648457,0.521177,-1.331600,0.687922,1.307501,linear_reg,,global
3,mean_nwe_lsfo_crack_trade_month,-0.604382,1.156572,-0.522563,0.604769,-2.957446,1.748682,1.726553,linear_reg,,global
4,mean_nwe_ulsfo_crack_trade_month_lag3,0.235534,2.701132,0.087198,0.931041,-5.259961,5.731028,18.792161,linear_reg,,global
5,mean_sing_gasoline_vs_vlsfo_trade_month,1.376711,1.314291,1.047494,0.302484,-1.297233,4.050656,1.620407,linear_reg,,global
6,mean_sing_vlsfo_crack_trade_month_lag3,0.754603,3.366884,0.224125,0.824042,-6.095375,7.604581,18.673177,linear_reg,,global
7,new_sweet_sr_margin,0.540160,2.280776,0.236832,0.814251,-4.100113,5.180434,1.990470,linear_reg,,global
8,totaltar,-0.331420,0.189424,-1.749618,0.089480,-0.716807,0.053966,2.753345,linear_reg,,global


,metric,value,split,model,model_group_name,group
0,rmse,30.620123,train,linear_reg,,global
1,mae,24.988846,train,linear_reg,,global
2,mape,30.63036,train,linear_reg,,global
3,smape,26.323762,train,linear_reg,,global
4,r_squared,0.24573,train,linear_reg,,global
5,mda,58.536585,train,linear_reg,,global
6,adj_r_squared,0.033591,train,linear_reg,,global
7,rmse,40.693426,test,linear_reg,,global
8,mae,33.265367,test,linear_reg,,global
9,mape,25.063394,test,linear_reg,,global


## 1.4 - OLS - sklearn

In [15]:
# sklearn engine (default)
spec_sk = linear_reg()
fit_sk = spec_sk.fit(train_data, FORMULA_STR)
fit_sk = fit_sk.evaluate(test_data)

print("sklearn model fitted successfully!")

outputs_sk, coefs_sk, stats_sk = fit_sk.extract_outputs()
display(outputs_sk)
display(coefs_sk)
display(stats_sk)

fig = plot_forecast(fit_sk, title="Sales Forecast")
fig.show()

sklearn model fitted successfully!


,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,100.399274,137.65,37.250726,train,linear_reg,,global
1,2020-05-01,113.53,98.415778,113.53,15.114222,train,linear_reg,,global
2,2020-06-01,43.31,102.319372,43.31,-59.009372,train,linear_reg,,global
3,2020-07-01,79.77,95.580130,79.77,-15.810130,train,linear_reg,,global
4,2020-08-01,48.27,89.088577,48.27,-40.818577,train,linear_reg,,global
5,2020-09-01,98.01,92.715549,98.01,5.294451,train,linear_reg,,global
6,2020-10-01,49.77,89.049033,49.77,-39.279033,train,linear_reg,,global
7,2020-11-01,35.48,85.125478,35.48,-49.645478,train,linear_reg,,global
8,2020-12-01,63.98,85.064946,63.98,-21.084946,train,linear_reg,,global
9,2021-01-01,109.76,88.002123,109.76,21.757877,train,linear_reg,,global


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,Intercept,86.200233,17.986252,4.792562,0.000034,49.606927,122.793538,NaN,linear_reg,,global
1,mean_med_diesel_crack_input1_trade_month_lag2,-0.062735,0.112253,-0.558873,0.580022,-0.291116,0.165645,1.240473,linear_reg,,global
2,mean_nwe_hsfo_crack_trade_month_lag1,-0.321839,0.496315,-0.648457,0.521177,-1.331600,0.687922,1.307501,linear_reg,,global
3,mean_nwe_lsfo_crack_trade_month,-0.604382,1.156572,-0.522563,0.604769,-2.957446,1.748682,1.726553,linear_reg,,global
4,mean_nwe_ulsfo_crack_trade_month_lag3,0.235534,2.701132,0.087198,0.931041,-5.259961,5.731028,18.792161,linear_reg,,global
5,mean_sing_gasoline_vs_vlsfo_trade_month,1.376711,1.314291,1.047494,0.302484,-1.297233,4.050656,1.620407,linear_reg,,global
6,mean_sing_vlsfo_crack_trade_month_lag3,0.754603,3.366884,0.224125,0.824042,-6.095375,7.604581,18.673177,linear_reg,,global
7,new_sweet_sr_margin,0.540160,2.280776,0.236832,0.814251,-4.100113,5.180434,1.990470,linear_reg,,global
8,totaltar,-0.331420,0.189424,-1.749618,0.089480,-0.716807,0.053966,2.753345,linear_reg,,global


,metric,value,split,model,model_group_name,group
0,rmse,30.620123,train,linear_reg,,global
1,mae,24.988846,train,linear_reg,,global
2,mape,30.63036,train,linear_reg,,global
3,smape,26.323762,train,linear_reg,,global
4,r_squared,0.24573,train,linear_reg,,global
5,mda,58.536585,train,linear_reg,,global
6,adj_r_squared,0.062877,train,linear_reg,,global
7,rmse,40.693426,test,linear_reg,,global
8,mae,33.265367,test,linear_reg,,global
9,mape,25.063394,test,linear_reg,,global


In [16]:
from py_parsnip import linear_reg

# Create specification with statsmodels engine
spec_elasticnet = linear_reg(penalty = 0.1, mixture = 0.5).set_engine("sklearn")

# Fit model
fit_elasticnet = spec_elasticnet.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_elasticnet = fit_elasticnet.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs from both models
outputs_elasticnet, coefs_elasticnet, stats_elasticnet = fit_elasticnet.extract_outputs()

display(outputs_elasticnet)
display(coefs_elasticnet)
display(stats_elasticnet)

fig = plot_forecast(fit_elasticnet, title="Sales Forecast")
fig.show()

Model fitted successfully!


,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,100.383020,137.65,37.266980,train,linear_reg,,global
1,2020-05-01,113.53,98.401218,113.53,15.128782,train,linear_reg,,global
2,2020-06-01,43.31,102.279842,43.31,-58.969842,train,linear_reg,,global
3,2020-07-01,79.77,95.600821,79.77,-15.830821,train,linear_reg,,global
4,2020-08-01,48.27,89.106229,48.27,-40.836229,train,linear_reg,,global
5,2020-09-01,98.01,92.755125,98.01,5.254875,train,linear_reg,,global
6,2020-10-01,49.77,89.122693,49.77,-39.352693,train,linear_reg,,global
7,2020-11-01,35.48,85.198109,35.48,-49.718109,train,linear_reg,,global
8,2020-12-01,63.98,85.125540,63.98,-21.145540,train,linear_reg,,global
9,2021-01-01,109.76,88.045472,109.76,21.714528,train,linear_reg,,global


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,Intercept,86.221134,NaN,NaN,NaN,NaN,NaN,NaN,linear_reg,,global
1,mean_med_diesel_crack_input1_trade_month_lag2,-0.063024,NaN,NaN,NaN,NaN,NaN,NaN,linear_reg,,global
2,mean_nwe_hsfo_crack_trade_month_lag1,-0.321231,NaN,NaN,NaN,NaN,NaN,NaN,linear_reg,,global
3,mean_nwe_lsfo_crack_trade_month,-0.602413,NaN,NaN,NaN,NaN,NaN,NaN,linear_reg,,global
4,mean_nwe_ulsfo_crack_trade_month_lag3,0.248361,NaN,NaN,NaN,NaN,NaN,NaN,linear_reg,,global
5,mean_sing_gasoline_vs_vlsfo_trade_month,1.367637,NaN,NaN,NaN,NaN,NaN,NaN,linear_reg,,global
6,mean_sing_vlsfo_crack_trade_month_lag3,0.734150,NaN,NaN,NaN,NaN,NaN,NaN,linear_reg,,global
7,new_sweet_sr_margin,0.534600,NaN,NaN,NaN,NaN,NaN,NaN,linear_reg,,global
8,totaltar,-0.331311,NaN,NaN,NaN,NaN,NaN,NaN,linear_reg,,global


,metric,value,split,model,model_group_name,group
0,rmse,30.620167,train,linear_reg,,global
1,mae,24.985282,train,linear_reg,,global
2,mape,30.626065,train,linear_reg,,global
3,smape,26.317628,train,linear_reg,,global
4,r_squared,0.245728,train,linear_reg,,global
5,mda,58.536585,train,linear_reg,,global
6,adj_r_squared,0.062874,train,linear_reg,,global
7,rmse,40.688897,test,linear_reg,,global
8,mae,33.275254,test,linear_reg,,global
9,mape,25.069597,test,linear_reg,,global


In [17]:
from py_parsnip import linear_reg

# Create specification with statsmodels engine
spec_ridge  = linear_reg(penalty = 0.1, mixture = 0).set_engine("sklearn")

# Fit model
fit_ridge = spec_ridge.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_ridge = fit_ridge.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs from both models
outputs_ridge, coefs_ridge, stats_ridge = fit_ridge.extract_outputs()

display(outputs_ridge)
display(coefs_ridge)
display(stats_ridge)

fig = plot_forecast(fit_ridge, title="Sales Forecast")
fig.show()

Model fitted successfully!


,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,100.398731,137.65,37.251269,train,linear_reg,,global
1,2020-05-01,113.53,98.415512,113.53,15.114488,train,linear_reg,,global
2,2020-06-01,43.31,102.318917,43.31,-59.008917,train,linear_reg,,global
3,2020-07-01,79.77,95.580562,79.77,-15.810562,train,linear_reg,,global
4,2020-08-01,48.27,89.088491,48.27,-40.818491,train,linear_reg,,global
5,2020-09-01,98.01,92.716416,98.01,5.293584,train,linear_reg,,global
6,2020-10-01,49.77,89.050808,49.77,-39.280808,train,linear_reg,,global
7,2020-11-01,35.48,85.127188,35.48,-49.647188,train,linear_reg,,global
8,2020-12-01,63.98,85.066534,63.98,-21.086534,train,linear_reg,,global
9,2021-01-01,109.76,88.003213,109.76,21.756787,train,linear_reg,,global


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,Intercept,86.200726,NaN,NaN,NaN,NaN,NaN,NaN,linear_reg,,global
1,mean_med_diesel_crack_input1_trade_month_lag2,-0.062744,NaN,NaN,NaN,NaN,NaN,NaN,linear_reg,,global
2,mean_nwe_hsfo_crack_trade_month_lag1,-0.321818,NaN,NaN,NaN,NaN,NaN,NaN,linear_reg,,global
3,mean_nwe_lsfo_crack_trade_month,-0.604312,NaN,NaN,NaN,NaN,NaN,NaN,linear_reg,,global
4,mean_nwe_ulsfo_crack_trade_month_lag3,0.235968,NaN,NaN,NaN,NaN,NaN,NaN,linear_reg,,global
5,mean_sing_gasoline_vs_vlsfo_trade_month,1.376470,NaN,NaN,NaN,NaN,NaN,NaN,linear_reg,,global
6,mean_sing_vlsfo_crack_trade_month_lag3,0.753959,NaN,NaN,NaN,NaN,NaN,NaN,linear_reg,,global
7,new_sweet_sr_margin,0.540106,NaN,NaN,NaN,NaN,NaN,NaN,linear_reg,,global
8,totaltar,-0.331416,NaN,NaN,NaN,NaN,NaN,NaN,linear_reg,,global


,metric,value,split,model,model_group_name,group
0,rmse,30.620123,train,linear_reg,,global
1,mae,24.988759,train,linear_reg,,global
2,mape,30.630237,train,linear_reg,,global
3,smape,26.323602,train,linear_reg,,global
4,r_squared,0.24573,train,linear_reg,,global
5,mda,58.536585,train,linear_reg,,global
6,adj_r_squared,0.062877,train,linear_reg,,global
7,rmse,40.693363,test,linear_reg,,global
8,mae,33.265777,test,linear_reg,,global
9,mape,25.06369,test,linear_reg,,global


In [18]:
from py_parsnip import linear_reg

# Create specification with statsmodels engine
spec_lasso  = linear_reg(penalty = 0.1, mixture = 1).set_engine("sklearn")

# Fit model
fit_lasso = spec_lasso.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_lasso = fit_lasso.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs from both models
outputs_lasso, coefs_lasso, stats_lasso = fit_lasso.extract_outputs()

display(outputs_lasso)
display(coefs_lasso)
display(stats_lasso)

fig = plot_forecast(fit_lasso, title="Sales Forecast")
fig.show()

Model fitted successfully!


,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,100.386531,137.65,37.263469,train,linear_reg,,global
1,2020-05-01,113.53,98.397047,113.53,15.132953,train,linear_reg,,global
2,2020-06-01,43.31,102.259763,43.31,-58.949763,train,linear_reg,,global
3,2020-07-01,79.77,95.603816,79.77,-15.833816,train,linear_reg,,global
4,2020-08-01,48.27,89.125526,48.27,-40.855526,train,linear_reg,,global
5,2020-09-01,98.01,92.758953,98.01,5.251047,train,linear_reg,,global
6,2020-10-01,49.77,89.123522,49.77,-39.353522,train,linear_reg,,global
7,2020-11-01,35.48,85.199745,35.48,-49.719745,train,linear_reg,,global
8,2020-12-01,63.98,85.120346,63.98,-21.140346,train,linear_reg,,global
9,2021-01-01,109.76,88.043265,109.76,21.716735,train,linear_reg,,global


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,Intercept,86.222347,NaN,NaN,NaN,NaN,NaN,NaN,linear_reg,,global
1,mean_med_diesel_crack_input1_trade_month_lag2,-0.062958,NaN,NaN,NaN,NaN,NaN,NaN,linear_reg,,global
2,mean_nwe_hsfo_crack_trade_month_lag1,-0.321461,NaN,NaN,NaN,NaN,NaN,NaN,linear_reg,,global
3,mean_nwe_lsfo_crack_trade_month,-0.603258,NaN,NaN,NaN,NaN,NaN,NaN,linear_reg,,global
4,mean_nwe_ulsfo_crack_trade_month_lag3,0.243773,NaN,NaN,NaN,NaN,NaN,NaN,linear_reg,,global
5,mean_sing_gasoline_vs_vlsfo_trade_month,1.368613,NaN,NaN,NaN,NaN,NaN,NaN,linear_reg,,global
6,mean_sing_vlsfo_crack_trade_month_lag3,0.739691,NaN,NaN,NaN,NaN,NaN,NaN,linear_reg,,global
7,new_sweet_sr_margin,0.531407,NaN,NaN,NaN,NaN,NaN,NaN,linear_reg,,global
8,totaltar,-0.331394,NaN,NaN,NaN,NaN,NaN,NaN,linear_reg,,global


,metric,value,split,model,model_group_name,group
0,rmse,30.620165,train,linear_reg,,global
1,mae,24.985312,train,linear_reg,,global
2,mape,30.626764,train,linear_reg,,global
3,smape,26.318093,train,linear_reg,,global
4,r_squared,0.245728,train,linear_reg,,global
5,mda,58.536585,train,linear_reg,,global
6,adj_r_squared,0.062874,train,linear_reg,,global
7,rmse,40.687277,test,linear_reg,,global
8,mae,33.269156,test,linear_reg,,global
9,mape,25.064423,test,linear_reg,,global


In [19]:
from py_parsnip import gen_additive_mod

# Create specification with statsmodels engine
spec_gam  = gen_additive_mod()

# Fit model
fit_gam = spec_gam.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_gam = fit_gam.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs from both models
outputs_gam, coefs_gam, stats_gam = fit_gam.extract_outputs()
display(outputs_gam)
display(coefs_gam)
display(stats_gam)

fig = plot_forecast(fit_gam, title="Sales Forecast")
fig.show()

Model fitted successfully!


,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,99.853995,137.65,37.796005,train,gen_additive_mod,,global
1,2020-05-01,113.53,100.402512,113.53,13.127488,train,gen_additive_mod,,global
2,2020-06-01,43.31,82.301749,43.31,-38.991749,train,gen_additive_mod,,global
3,2020-07-01,79.77,81.837073,79.77,-2.067073,train,gen_additive_mod,,global
4,2020-08-01,48.27,70.478954,48.27,-22.208954,train,gen_additive_mod,,global
5,2020-09-01,98.01,78.236004,98.01,19.773996,train,gen_additive_mod,,global
6,2020-10-01,49.77,76.597322,49.77,-26.827322,train,gen_additive_mod,,global
7,2020-11-01,35.48,80.898126,35.48,-45.418126,train,gen_additive_mod,,global
8,2020-12-01,63.98,83.775675,63.98,-19.795675,train,gen_additive_mod,,global
9,2021-01-01,109.76,90.031667,109.76,19.728333,train,gen_additive_mod,,global


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,Intercept,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,gen_additive_mod,,global
1,mean_med_diesel_crack_input1_trade_month_lag2,19.989337,NaN,NaN,NaN,NaN,NaN,NaN,gen_additive_mod,,global
2,mean_nwe_hsfo_crack_trade_month_lag1,14.145334,NaN,NaN,NaN,NaN,NaN,NaN,gen_additive_mod,,global
3,mean_nwe_lsfo_crack_trade_month,16.685539,NaN,NaN,NaN,NaN,NaN,NaN,gen_additive_mod,,global
4,mean_nwe_ulsfo_crack_trade_month_lag3,13.340001,NaN,NaN,NaN,NaN,NaN,NaN,gen_additive_mod,,global
5,mean_sing_gasoline_vs_vlsfo_trade_month,10.944696,NaN,NaN,NaN,NaN,NaN,NaN,gen_additive_mod,,global
6,mean_sing_vlsfo_crack_trade_month_lag3,13.678626,NaN,NaN,NaN,NaN,NaN,NaN,gen_additive_mod,,global
7,new_sweet_sr_margin,17.976755,NaN,NaN,NaN,NaN,NaN,NaN,gen_additive_mod,,global
8,totaltar,20.995090,NaN,NaN,NaN,NaN,NaN,NaN,gen_additive_mod,,global


,metric,value,split,model,model_group_name,group
0,rmse,26.668252,train,gen_additive_mod,,global
1,mae,21.795943,train,gen_additive_mod,,global
2,mape,26.600686,train,gen_additive_mod,,global
3,smape,23.352229,train,gen_additive_mod,,global
4,r_squared,0.42786,train,gen_additive_mod,,global
5,mda,56.097561,train,gen_additive_mod,,global
6,adj_r_squared,0.269313,train,gen_additive_mod,,global
7,rmse,31.674012,test,gen_additive_mod,,global
8,mae,25.568623,test,gen_additive_mod,,global
9,mape,19.164949,test,gen_additive_mod,,global


In [20]:
# from py_parsnip import arima_reg

# # Create specification with statsmodels engine
# spec_auto_arima  = arima_reg().set_engine("auto_arima")

# # Fit model
# fit_auto_arima = spec_auto_arima.fit(train_data, FORMULA_STR)

# # Evaluate on test set
# fit_auto_arima = fit_auto_arima.evaluate(test_data)

# print("Model fitted successfully!")

# # Extract outputs from both models
# outputs_auto_arima, coefs_auto_arima, stats_auto_arima = fit_auto_arima.extract_outputs()
# display(outputs_auto_arima)
# display(coefs_auto_arima)
# display(stats_auto_arima)

In [21]:
from py_parsnip import prophet_reg

# Create specification with statsmodels engine
spec_prophet  = prophet_reg()

# Fit model
fit_prophet = spec_prophet.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_prophet = fit_prophet.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs from both models
outputs_prophet, coefs_prophet, stats_prophet = fit_prophet.extract_outputs()
display(outputs_prophet)
display(coefs_prophet)
display(stats_prophet)

fig = plot_forecast(fit_prophet, title="Sales Forecast")
fig.show()

Model fitted successfully!


,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,115.944878,137.65,21.705122,train,prophet_reg,,global
1,2020-05-01,113.53,118.598941,113.53,-5.068941,train,prophet_reg,,global
2,2020-06-01,43.31,68.304292,43.31,-24.994292,train,prophet_reg,,global
3,2020-07-01,79.77,72.219618,79.77,7.550382,train,prophet_reg,,global
4,2020-08-01,48.27,44.355810,48.27,3.914190,train,prophet_reg,,global
5,2020-09-01,98.01,110.773505,98.01,-12.763505,train,prophet_reg,,global
6,2020-10-01,49.77,70.704920,49.77,-20.934920,train,prophet_reg,,global
7,2020-11-01,35.48,38.467477,35.48,-2.987477,train,prophet_reg,,global
8,2020-12-01,63.98,88.571625,63.98,-24.591625,train,prophet_reg,,global
9,2021-01-01,109.76,116.650937,109.76,-6.890937,train,prophet_reg,,global


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,growth,linear,NaN,NaN,NaN,NaN,NaN,NaN,prophet_reg,,global
1,changepoint_prior_scale,0.05,NaN,NaN,NaN,NaN,NaN,NaN,prophet_reg,,global
2,seasonality_prior_scale,10.0,NaN,NaN,NaN,NaN,NaN,NaN,prophet_reg,,global
3,seasonality_mode,additive,NaN,NaN,NaN,NaN,NaN,NaN,prophet_reg,,global
4,n_changepoints,25,NaN,NaN,NaN,NaN,NaN,NaN,prophet_reg,,global


,metric,value,split,model,model_group_name,group
0,rmse,17.969607,train,prophet_reg,,global
1,mae,13.879322,train,prophet_reg,,global
2,mape,15.11841,train,prophet_reg,,global
3,smape,14.423303,train,prophet_reg,,global
4,r_squared,0.740229,train,prophet_reg,,global
5,mda,68.292683,train,prophet_reg,,global
6,rmse,79.938047,test,prophet_reg,,global
7,mae,71.511399,test,prophet_reg,,global
8,mape,57.202044,test,prophet_reg,,global
9,smape,85.011131,test,prophet_reg,,global


In [22]:
from py_parsnip import exp_smoothing

# Create specification with statsmodels engine
spec_exp_smoothing  = exp_smoothing(error="add", trend="add", season="add", seasonal_period=12)

# Fit model
fit_exp_smoothing = spec_exp_smoothing.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_exp_smoothing = fit_exp_smoothing.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs from both models
outputs_exp_smoothing, coefs_exp_smoothing, stats_exp_smoothing = fit_exp_smoothing.extract_outputs()
display(outputs_exp_smoothing)
display(coefs_exp_smoothing)
display(stats_exp_smoothing)

fig = plot_forecast(fit_exp_smoothing, title="Sales Forecast")
fig.show()

Model fitted successfully!


/Users/matthewdeane/Documents/Data Science/python/_projects/py-tidymodels/py-tidymodels2/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.



,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,119.568266,137.65,18.081734,train,exp_smoothing,,global
1,2020-05-01,113.53,102.034604,113.53,11.495396,train,exp_smoothing,,global
2,2020-06-01,43.31,72.949596,43.31,-29.639596,train,exp_smoothing,,global
3,2020-07-01,79.77,90.679617,79.77,-10.909617,train,exp_smoothing,,global
4,2020-08-01,48.27,78.521668,48.27,-30.251668,train,exp_smoothing,,global
5,2020-09-01,98.01,69.405629,98.01,28.604371,train,exp_smoothing,,global
6,2020-10-01,49.77,61.577041,49.77,-11.807041,train,exp_smoothing,,global
7,2020-11-01,35.48,89.866576,35.48,-54.386576,train,exp_smoothing,,global
8,2020-12-01,63.98,112.668013,63.98,-48.688013,train,exp_smoothing,,global
9,2021-01-01,109.76,94.222961,109.76,15.537039,train,exp_smoothing,,global


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,alpha (smoothing_level),1.490116e-08,NaN,NaN,NaN,NaN,NaN,NaN,exp_smoothing,,global
1,beta (smoothing_trend),0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,exp_smoothing,,global
2,gamma (smoothing_seasonal),0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,exp_smoothing,,global
3,phi (damping_trend),NaN,NaN,NaN,NaN,NaN,NaN,NaN,exp_smoothing,,global
4,l0 (initial_level),8.749568e+01,NaN,NaN,NaN,NaN,NaN,NaN,exp_smoothing,,global
5,b0 (initial_trend),4.893895e-01,NaN,NaN,NaN,NaN,NaN,NaN,exp_smoothing,,global


,metric,value,split,model,model_group_name,group
0,rmse,29.174087,train,exp_smoothing,,global
1,mae,24.100522,train,exp_smoothing,,global
2,mape,32.149773,train,exp_smoothing,,global
3,smape,26.919124,train,exp_smoothing,,global
4,r_squared,0.315289,train,exp_smoothing,,global
5,mda,68.292683,train,exp_smoothing,,global
6,rmse,26.522722,test,exp_smoothing,,global
7,mae,20.040026,test,exp_smoothing,,global
8,mape,15.517037,test,exp_smoothing,,global
9,smape,17.038668,test,exp_smoothing,,global


In [23]:
from py_parsnip import arima_boost

# Create specification with statsmodels engine
spec_arima_boost  = arima_boost()

# Fit model
fit_arima_boost = spec_arima_boost.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_arima_boost = fit_arima_boost.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs from both models
outputs_arima_boost, coefs_arima_boost, stats_arima_boost = fit_arima_boost.extract_outputs()
display(outputs_arima_boost)
display(coefs_arima_boost)
display(stats_arima_boost)

fig = plot_forecast(fit_arima_boost, title="Sales Forecast")
fig.show()

Model fitted successfully!


/Users/matthewdeane/Documents/Data Science/python/_projects/py-tidymodels/py-tidymodels2/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

/Users/matthewdeane/Documents/Data Science/python/_projects/py-tidymodels/py-tidymodels2/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.



,date,actuals,arima_fitted,xgb_fitted,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,169.325235,-31.621536,137.703698,137.65,-0.053698,train,arima_boost,,global
1,2020-05-01,113.53,124.427639,-10.762588,113.665052,113.53,-0.135052,train,arima_boost,,global
2,2020-06-01,43.31,54.173928,-10.796356,43.377572,43.31,-0.067572,train,arima_boost,,global
3,2020-07-01,79.77,70.053759,9.858168,79.911927,79.77,-0.141927,train,arima_boost,,global
4,2020-08-01,48.27,64.287665,-15.925306,48.362358,48.27,-0.092358,train,arima_boost,,global
5,2020-09-01,98.01,60.818109,36.859722,97.677831,98.01,0.332169,train,arima_boost,,global
6,2020-10-01,49.77,60.350545,-10.485323,49.865222,49.77,-0.095222,train,arima_boost,,global
7,2020-11-01,35.48,60.452955,-24.597380,35.855575,35.48,-0.375575,train,arima_boost,,global
8,2020-12-01,63.98,43.504698,20.431410,63.936108,63.98,0.043892,train,arima_boost,,global
9,2021-01-01,109.76,58.521470,51.286575,109.808046,109.76,-0.048046,train,arima_boost,,global


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,arima_mean_med_diesel_crack_input1_trade_month...,-0.293925,0.161884,-1.815648,0.069424,NaN,NaN,NaN,arima_boost,,global
1,arima_mean_nwe_hsfo_crack_trade_month_lag1,-1.180698,0.601245,-1.963757,0.049558,NaN,NaN,NaN,arima_boost,,global
2,arima_mean_nwe_lsfo_crack_trade_month,-0.945473,1.429324,-0.661483,0.508303,NaN,NaN,NaN,arima_boost,,global
3,arima_mean_nwe_ulsfo_crack_trade_month_lag3,-0.742078,3.252441,-0.228160,0.819522,NaN,NaN,NaN,arima_boost,,global
4,arima_mean_sing_gasoline_vs_vlsfo_trade_month,0.954441,1.844654,0.517410,0.604870,NaN,NaN,NaN,arima_boost,,global
5,arima_mean_sing_vlsfo_crack_trade_month_lag3,6.501104,4.018757,1.617690,0.105729,NaN,NaN,NaN,arima_boost,,global
6,arima_new_sweet_sr_margin,-4.179990,2.840690,-1.471470,0.141164,NaN,NaN,NaN,arima_boost,,global
7,arima_totaltar,-0.884710,0.700530,-1.262915,0.206620,NaN,NaN,NaN,arima_boost,,global
8,arima_sigma2,1611.981189,502.974440,3.204897,0.001351,NaN,NaN,NaN,arima_boost,,global
9,xgb_n_estimators,100.000000,NaN,NaN,NaN,NaN,NaN,NaN,arima_boost,,global


,metric,value,split,model,model_group_name,group
0,rmse,0.193613,train,arima_boost,,global
1,mae,0.14306,train,arima_boost,,global
2,mape,0.162758,train,arima_boost,,global
3,smape,0.162595,train,arima_boost,,global
4,r_squared,0.99997,train,arima_boost,,global
5,mda,97.560976,train,arima_boost,,global
6,rmse,69.423571,test,arima_boost,,global
7,mae,62.170065,test,arima_boost,,global
8,mape,48.474816,test,arima_boost,,global
9,smape,68.572819,test,arima_boost,,global


In [24]:
from py_parsnip import prophet_boost

# Create specification with statsmodels engine
spec_prophet_boost  = prophet_boost(     
      seasonality_yearly=False,
      seasonality_weekly=False,
      seasonality_daily=False)   

# Fit model
fit_prophet_boost = spec_prophet_boost.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_prophet_boost = fit_prophet_boost.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs from both models
outputs_prophet_boost, coefs_prophet_boost, stats_prophet_boost = fit_prophet_boost.extract_outputs()
display(outputs_prophet_boost)
display(coefs_prophet_boost)
display(stats_prophet_boost)

fig = plot_forecast(fit_prophet_boost, title="Sales Forecast")
fig.show()

Model fitted successfully!


,date,actuals,prophet_fitted,xgb_fitted,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,112.553041,25.031872,137.584913,137.65,0.065087,train,prophet_boost,,global
1,2020-05-01,113.53,122.000742,-8.456347,113.544396,113.53,-0.014396,train,prophet_boost,,global
2,2020-06-01,43.31,70.450176,-27.126514,43.323661,43.31,-0.013661,train,prophet_boost,,global
3,2020-07-01,79.77,71.641046,8.101948,79.742994,79.77,0.027006,train,prophet_boost,,global
4,2020-08-01,48.27,59.675524,-11.405581,48.269943,48.27,0.000057,train,prophet_boost,,global
5,2020-09-01,98.01,109.218005,-11.201484,98.016521,98.01,-0.006521,train,prophet_boost,,global
6,2020-10-01,49.77,75.128970,-25.296751,49.832219,49.77,-0.062219,train,prophet_boost,,global
7,2020-11-01,35.48,55.915862,-20.444035,35.471828,35.48,0.008172,train,prophet_boost,,global
8,2020-12-01,63.98,96.949583,-32.942497,64.007086,63.98,-0.027086,train,prophet_boost,,global
9,2021-01-01,109.76,116.987087,-7.255165,109.731922,109.76,0.028078,train,prophet_boost,,global


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,prophet_growth,linear,NaN,NaN,NaN,NaN,NaN,NaN,prophet_boost,,global
1,prophet_changepoint_prior_scale,0.05,NaN,NaN,NaN,NaN,NaN,NaN,prophet_boost,,global
2,prophet_seasonality_prior_scale,10.0,NaN,NaN,NaN,NaN,NaN,NaN,prophet_boost,,global
3,prophet_seasonality_mode,additive,NaN,NaN,NaN,NaN,NaN,NaN,prophet_boost,,global
4,prophet_n_changepoints,25,NaN,NaN,NaN,NaN,NaN,NaN,prophet_boost,,global
5,prophet_changepoint_range,0.8,NaN,NaN,NaN,NaN,NaN,NaN,prophet_boost,,global
6,prophet_n_changepoints_detected,25,NaN,NaN,NaN,NaN,NaN,NaN,prophet_boost,,global
7,xgb_n_estimators,100,NaN,NaN,NaN,NaN,NaN,NaN,prophet_boost,,global
8,xgb_max_depth,6,NaN,NaN,NaN,NaN,NaN,NaN,prophet_boost,,global
9,xgb_learning_rate,0.1,NaN,NaN,NaN,NaN,NaN,NaN,prophet_boost,,global


,metric,value,split,model,model_group_name,group
0,rmse,0.083952,train,prophet_boost,,global
1,mae,0.046208,train,prophet_boost,,global
2,mape,0.049346,train,prophet_boost,,global
3,smape,0.049325,train,prophet_boost,,global
4,r_squared,0.999994,train,prophet_boost,,global
5,mda,97.560976,train,prophet_boost,,global
6,rmse,46.769635,test,prophet_boost,,global
7,mae,40.701818,test,prophet_boost,,global
8,mape,32.799138,test,prophet_boost,,global
9,smape,38.398995,test,prophet_boost,,global


In [25]:
from py_parsnip import boost_tree   

# Create specification with statsmodels engine
spec_boost_tree  = boost_tree().set_engine("lightgbm")

# Fit model
fit_boost_tree = spec_boost_tree.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_boost_tree = fit_boost_tree.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs from both models
outputs_boost_tree, coefs_boost_tree, stats_boost_tree = fit_boost_tree.extract_outputs()
display(outputs_boost_tree)
display(coefs_boost_tree)
display(stats_boost_tree)

fig = plot_forecast(fit_boost_tree, title="Sales Forecast")
fig.show()

Model fitted successfully!


,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,97.509274,137.65,40.140726,train,boost_tree,,global
1,2020-05-01,113.53,95.454031,113.53,18.075969,train,boost_tree,,global
2,2020-06-01,43.31,89.629161,43.31,-46.319161,train,boost_tree,,global
3,2020-07-01,79.77,86.321974,79.77,-6.551974,train,boost_tree,,global
4,2020-08-01,48.27,70.935966,48.27,-22.665966,train,boost_tree,,global
5,2020-09-01,98.01,92.146843,98.01,5.863157,train,boost_tree,,global
6,2020-10-01,49.77,76.760835,49.77,-26.990835,train,boost_tree,,global
7,2020-11-01,35.48,76.760835,35.48,-41.280835,train,boost_tree,,global
8,2020-12-01,63.98,99.807684,63.98,-35.827684,train,boost_tree,,global
9,2021-01-01,109.76,99.807684,109.76,9.952316,train,boost_tree,,global


,variable,importance,model,model_group_name,group
0,new_sweet_sr_margin,29,boost_tree,,global
1,mean_sing_gasoline_vs_vlsfo_trade_month,17,boost_tree,,global
2,mean_nwe_hsfo_crack_trade_month_lag1,16,boost_tree,,global
3,mean_nwe_lsfo_crack_trade_month,14,boost_tree,,global
4,mean_med_diesel_crack_input1_trade_month_lag2,13,boost_tree,,global
5,mean_sing_vlsfo_crack_trade_month_lag3,11,boost_tree,,global
6,Intercept,0,boost_tree,,global
7,mean_nwe_ulsfo_crack_trade_month_lag3,0,boost_tree,,global
8,totaltar,0,boost_tree,,global


,metric,value,split,model,model_group_name,group
0,rmse,30.978337,train,boost_tree,,global
1,mae,25.755464,train,boost_tree,,global
2,mape,33.562416,train,boost_tree,,global
3,smape,28.029345,train,boost_tree,,global
4,r_squared,0.227979,train,boost_tree,,global
5,mda,48.780488,train,boost_tree,,global
6,rmse,31.623978,test,boost_tree,,global
7,mae,25.28719,test,boost_tree,,global
8,mape,19.096433,test,boost_tree,,global
9,smape,21.908452,test,boost_tree,,global


In [26]:
from py_parsnip import null_model

# Create specification with statsmodels engine
spec_null_model  = null_model(strategy="mean")

# Fit model
fit_null_model = spec_null_model.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_null_model = fit_null_model.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs from both models
outputs_null_model, coefs_null_model, stats_null_model = fit_null_model.extract_outputs()
display(outputs_null_model)
display(coefs_null_model)
display(stats_null_model)

fig = plot_forecast(fit_null_model, title="Sales Forecast")
fig.show()

Model fitted successfully!


,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,99.005,99.005,38.645,train,null_model,,global
1,2020-05-01,113.53,99.005,99.005,14.525,train,null_model,,global
2,2020-06-01,43.31,99.005,99.005,-55.695,train,null_model,,global
3,2020-07-01,79.77,99.005,99.005,-19.235,train,null_model,,global
4,2020-08-01,48.27,99.005,99.005,-50.735,train,null_model,,global
5,2020-09-01,98.01,99.005,99.005,-0.995,train,null_model,,global
6,2020-10-01,49.77,99.005,99.005,-49.235,train,null_model,,global
7,2020-11-01,35.48,99.005,99.005,-63.525,train,null_model,,global
8,2020-12-01,63.98,99.005,99.005,-35.025,train,null_model,,global
9,2021-01-01,109.76,99.005,99.005,10.755,train,null_model,,global


,variable,coefficient,std_error,p_value,ci_0.025,ci_0.975,model,model_group_name,group
0,(Intercept),99.005,NaN,NaN,NaN,NaN,null_model,,global


,metric,value,split,model,model_group_name,group
0,rmse,35.256848,train,null_model,,global
1,mae,27.009762,train,null_model,,global
2,mape,38.484528,train,null_model,,global
3,r_squared,0.0,train,null_model,,global
4,baseline_value,99.005,train,null_model,,global
5,rmse,32.591708,test,null_model,,global
6,mae,25.992333,test,null_model,,global
7,mape,19.028925,test,null_model,,global
8,r_squared,-1.747465,test,null_model,,global
9,train_start_date,2020-04-01T00:00:00.000000000,train,null_model,,global


In [27]:
from py_parsnip import naive_reg

# Create specification with statsmodels engine
spec_naive_model  = naive_reg(strategy="naive")

# Fit model
fit_naive_model = spec_naive_model.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_naive_model = fit_naive_model.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs from both models
outputs_naive_model, coefs_naive_model, stats_naive_model = fit_naive_model.extract_outputs()
display(outputs_naive_model)
display(coefs_naive_model)
display(stats_naive_model)

fig = plot_forecast(fit_naive_model, title="Sales Forecast")
fig.show()

Model fitted successfully!


,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,137.65,137.65,0.00,train,naive_reg,,global
1,2020-05-01,113.53,137.65,137.65,-24.12,train,naive_reg,,global
2,2020-06-01,43.31,113.53,113.53,-70.22,train,naive_reg,,global
3,2020-07-01,79.77,43.31,43.31,36.46,train,naive_reg,,global
4,2020-08-01,48.27,79.77,79.77,-31.50,train,naive_reg,,global
5,2020-09-01,98.01,48.27,48.27,49.74,train,naive_reg,,global
6,2020-10-01,49.77,98.01,98.01,-48.24,train,naive_reg,,global
7,2020-11-01,35.48,49.77,49.77,-14.29,train,naive_reg,,global
8,2020-12-01,63.98,35.48,35.48,28.50,train,naive_reg,,global
9,2021-01-01,109.76,63.98,63.98,45.78,train,naive_reg,,global


,variable,coefficient,std_error,p_value,ci_0.025,ci_0.975,model,model_group_name,group
0,strategy,naive,NaN,NaN,NaN,NaN,naive_reg,,global


,metric,value,split,model,model_group_name,group
0,rmse,39.867462,train,naive_reg,,global
1,mae,30.887619,train,naive_reg,,global
2,mape,38.600982,train,naive_reg,,global
3,r_squared,-0.278646,train,naive_reg,,global
4,strategy,naive,train,naive_reg,,global
5,rmse,69.468119,test,naive_reg,,global
6,mae,66.627333,test,naive_reg,,global
7,mape,52.262193,test,naive_reg,,global
8,r_squared,-11.482139,test,naive_reg,,global
9,train_start_date,2020-04-01T00:00:00.000000000,train,naive_reg,,global


In [28]:
from py_parsnip import naive_reg

# Create specification with statsmodels engine
spec_naive_model  = naive_reg(strategy="seasonal_naive", seasonal_period=12)

# Fit model
fit_naive_model = spec_naive_model.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_naive_model = fit_naive_model.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs from both models
outputs_naive_model, coefs_naive_model, stats_naive_model = fit_naive_model.extract_outputs()
display(outputs_naive_model)
display(coefs_naive_model)
display(stats_naive_model)

fig = plot_forecast(fit_naive_model, title="Sales Forecast")
fig.show()

Model fitted successfully!


,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,137.65,137.65,0.00,train,naive_reg,,global
1,2020-05-01,113.53,113.53,113.53,0.00,train,naive_reg,,global
2,2020-06-01,43.31,43.31,43.31,0.00,train,naive_reg,,global
3,2020-07-01,79.77,79.77,79.77,0.00,train,naive_reg,,global
4,2020-08-01,48.27,48.27,48.27,0.00,train,naive_reg,,global
5,2020-09-01,98.01,98.01,98.01,0.00,train,naive_reg,,global
6,2020-10-01,49.77,49.77,49.77,0.00,train,naive_reg,,global
7,2020-11-01,35.48,35.48,35.48,0.00,train,naive_reg,,global
8,2020-12-01,63.98,63.98,63.98,0.00,train,naive_reg,,global
9,2021-01-01,109.76,109.76,109.76,0.00,train,naive_reg,,global


,variable,coefficient,std_error,p_value,ci_0.025,ci_0.975,model,model_group_name,group
0,strategy,seasonal_naive,NaN,NaN,NaN,NaN,naive_reg,,global


,metric,value,split,model,model_group_name,group
0,rmse,42.541986,train,naive_reg,,global
1,mae,27.645714,train,naive_reg,,global
2,mape,33.286186,train,naive_reg,,global
3,r_squared,-0.455957,train,naive_reg,,global
4,strategy,seasonal_naive,train,naive_reg,,global
5,rmse,44.570802,test,naive_reg,,global
6,mae,31.266,test,naive_reg,,global
7,mape,24.715076,test,naive_reg,,global
8,r_squared,-4.138292,test,naive_reg,,global
9,train_start_date,2020-04-01T00:00:00.000000000,train,naive_reg,,global


In [29]:
from py_parsnip import naive_reg

# Create specification with statsmodels engine
spec_naive_model  = naive_reg(strategy="drift")

# Fit model
fit_naive_model = spec_naive_model.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_naive_model = fit_naive_model.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs from both models
outputs_naive_model, coefs_naive_model, stats_naive_model = fit_naive_model.extract_outputs()
display(outputs_naive_model)
display(coefs_naive_model)
display(stats_naive_model)

fig = plot_forecast(fit_naive_model, title="Sales Forecast")
fig.show()

Model fitted successfully!


,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,137.650000,137.650000,0.000000,train,naive_reg,,global
1,2020-05-01,113.53,135.716341,135.716341,-22.186341,train,naive_reg,,global
2,2020-06-01,43.31,111.596341,111.596341,-68.286341,train,naive_reg,,global
3,2020-07-01,79.77,41.376341,41.376341,38.393659,train,naive_reg,,global
4,2020-08-01,48.27,77.836341,77.836341,-29.566341,train,naive_reg,,global
5,2020-09-01,98.01,46.336341,46.336341,51.673659,train,naive_reg,,global
6,2020-10-01,49.77,96.076341,96.076341,-46.306341,train,naive_reg,,global
7,2020-11-01,35.48,47.836341,47.836341,-12.356341,train,naive_reg,,global
8,2020-12-01,63.98,33.546341,33.546341,30.433659,train,naive_reg,,global
9,2021-01-01,109.76,62.046341,62.046341,47.713659,train,naive_reg,,global


,variable,coefficient,std_error,p_value,ci_0.025,ci_0.975,model,model_group_name,group
0,strategy,drift,NaN,NaN,NaN,NaN,naive_reg,,global


,metric,value,split,model,model_group_name,group
0,rmse,39.821659,train,naive_reg,,global
1,mae,30.808722,train,naive_reg,,global
2,mape,38.155328,train,naive_reg,,global
3,r_squared,-0.275709,train,naive_reg,,global
4,strategy,drift,train,naive_reg,,global
5,rmse,84.801318,test,naive_reg,,global
6,mae,82.096602,test,naive_reg,,global
7,mape,64.940391,test,naive_reg,,global
8,r_squared,-17.600436,test,naive_reg,,global
9,train_start_date,2020-04-01T00:00:00.000000000,train,naive_reg,,global


In [30]:
from py_parsnip import null_model

# Create specification with statsmodels engine
spec_null_model  = null_model(strategy="mean")

# Fit model
fit_null_model = spec_null_model.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_null_model = fit_null_model.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs from both models
outputs_null_model, coefs_null_model, stats_null_model = fit_null_model.extract_outputs()
display(outputs_null_model)
display(coefs_null_model)
display(stats_null_model)

fig = plot_forecast(fit_null_model, title="Sales Forecast")
fig.show()

Model fitted successfully!


,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,99.005,99.005,38.645,train,null_model,,global
1,2020-05-01,113.53,99.005,99.005,14.525,train,null_model,,global
2,2020-06-01,43.31,99.005,99.005,-55.695,train,null_model,,global
3,2020-07-01,79.77,99.005,99.005,-19.235,train,null_model,,global
4,2020-08-01,48.27,99.005,99.005,-50.735,train,null_model,,global
5,2020-09-01,98.01,99.005,99.005,-0.995,train,null_model,,global
6,2020-10-01,49.77,99.005,99.005,-49.235,train,null_model,,global
7,2020-11-01,35.48,99.005,99.005,-63.525,train,null_model,,global
8,2020-12-01,63.98,99.005,99.005,-35.025,train,null_model,,global
9,2021-01-01,109.76,99.005,99.005,10.755,train,null_model,,global


,variable,coefficient,std_error,p_value,ci_0.025,ci_0.975,model,model_group_name,group
0,(Intercept),99.005,NaN,NaN,NaN,NaN,null_model,,global


,metric,value,split,model,model_group_name,group
0,rmse,35.256848,train,null_model,,global
1,mae,27.009762,train,null_model,,global
2,mape,38.484528,train,null_model,,global
3,r_squared,0.0,train,null_model,,global
4,baseline_value,99.005,train,null_model,,global
5,rmse,32.591708,test,null_model,,global
6,mae,25.992333,test,null_model,,global
7,mape,19.028925,test,null_model,,global
8,r_squared,-1.747465,test,null_model,,global
9,train_start_date,2020-04-01T00:00:00.000000000,train,null_model,,global


## Hybrid Model

Combine two models with different strategies (residual, sequential, weighted)

In [31]:
from py_parsnip import hybrid_model, linear_reg, rand_forest

# Create specification - combine linear regression with random forest
# Strategy: 'residual' trains model2 on residuals from model1
spec_hybrid_model = hybrid_model(
    model1=linear_reg(),
    model2=rand_forest().set_mode('regression'),
    strategy='residual'
)

# Fit model
fit_hybrid_model = spec_hybrid_model.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_hybrid_model = fit_hybrid_model.evaluate(test_data)

print("Hybrid model fitted successfully!")

# Extract outputs from hybrid model
outputs_hybrid_model, coefs_hybrid_model, stats_hybrid_model = fit_hybrid_model.extract_outputs()
display(outputs_hybrid_model)
display(coefs_hybrid_model)
display(stats_hybrid_model)

fig = plot_forecast(fit_hybrid_model, title="Sales Forecast")
fig.show()

Hybrid model fitted successfully!


,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,123.824229,137.65,13.825771,train,hybrid_model,,global
1,2020-05-01,113.53,115.971447,113.53,-2.441447,train,hybrid_model,,global
2,2020-06-01,43.31,65.323578,43.31,-22.013578,train,hybrid_model,,global
3,2020-07-01,79.77,86.887431,79.77,-7.117431,train,hybrid_model,,global
4,2020-08-01,48.27,61.794777,48.27,-13.524777,train,hybrid_model,,global
5,2020-09-01,98.01,82.044493,98.01,15.965507,train,hybrid_model,,global
6,2020-10-01,49.77,56.454429,49.77,-6.684429,train,hybrid_model,,global
7,2020-11-01,35.48,49.068755,35.48,-13.588755,train,hybrid_model,,global
8,2020-12-01,63.98,66.655547,63.98,-2.675547,train,hybrid_model,,global
9,2021-01-01,109.76,101.068411,109.76,8.691589,train,hybrid_model,,global


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,strategy,residual,NaN,NaN,NaN,NaN,NaN,NaN,hybrid_model,,global
1,model1_type,linear_reg,NaN,NaN,NaN,NaN,NaN,NaN,hybrid_model,,global
2,model2_type,rand_forest,NaN,NaN,NaN,NaN,NaN,NaN,hybrid_model,,global


,metric,value,split,model,model_group_name,group
0,rmse,12.34948,train,hybrid_model,,global
1,mae,9.808398,train,hybrid_model,,global
2,mape,11.959177,train,hybrid_model,,global
3,r_squared,0.87731,train,hybrid_model,,global
4,rmse,43.745194,test,hybrid_model,,global
5,mae,37.379446,test,hybrid_model,,global
6,mape,28.362202,test,hybrid_model,,global
7,r_squared,-3.949697,test,hybrid_model,,global
8,formula,target ~ mean_med_diesel_crack_input1_trade_mo...,,hybrid_model,,global
9,model_type,hybrid_model,,hybrid_model,,global


## Manual Regression

Specify coefficients manually for comparison with external forecasts or domain knowledge

In [32]:
from py_parsnip import manual_reg

# Create specification with manually specified coefficients
# Useful for comparing with external models or incorporating domain knowledge
spec_manual_reg = manual_reg(
    coefficients={"totaltar": 0.8, "mean_med_diesel_crack_input1_trade_month_lag2": 0.1, "mean_nwe_hsfo_crack_trade_month_lag1": 2},  # Example coefficients
    intercept=100.0
)

# Fit model (validates coefficients and calculates fitted values)
fit_manual_reg = spec_manual_reg.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_manual_reg = fit_manual_reg.evaluate(test_data)

print("Manual regression model fitted successfully!")

# Extract outputs from manual model
outputs_manual_reg, coefs_manual_reg, stats_manual_reg = fit_manual_reg.extract_outputs()
display(outputs_manual_reg)
display(coefs_manual_reg)
display(stats_manual_reg)

fig = plot_forecast(fit_manual_reg, title="Sales Forecast")
fig.show()

Manual regression model fitted successfully!


,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,137.65,71.522,137.65,66.128,train,manual_reg,,global
1,113.53,77.424,113.53,36.106,train,manual_reg,,global
2,43.31,79.819,43.31,-36.509,train,manual_reg,,global
3,79.77,83.394,79.77,-3.624,train,manual_reg,,global
4,48.27,84.400,48.27,-36.130,train,manual_reg,,global
5,98.01,84.584,98.01,13.426,train,manual_reg,,global
6,49.77,85.401,49.77,-35.631,train,manual_reg,,global
7,35.48,84.846,35.48,-49.366,train,manual_reg,,global
8,63.98,89.065,63.98,-25.085,train,manual_reg,,global
9,109.76,83.822,109.76,25.938,train,manual_reg,,global


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,Intercept,100.0,NaN,NaN,NaN,NaN,NaN,NaN,manual_reg,,global
1,mean_med_diesel_crack_input1_trade_month_lag2,0.1,NaN,NaN,NaN,NaN,NaN,NaN,manual_reg,,global
2,mean_nwe_hsfo_crack_trade_month_lag1,2.0,NaN,NaN,NaN,NaN,NaN,NaN,manual_reg,,global
3,mean_nwe_lsfo_crack_trade_month,0.0,NaN,NaN,NaN,NaN,NaN,NaN,manual_reg,,global
4,mean_nwe_ulsfo_crack_trade_month_lag3,0.0,NaN,NaN,NaN,NaN,NaN,NaN,manual_reg,,global
5,mean_sing_gasoline_vs_vlsfo_trade_month,0.0,NaN,NaN,NaN,NaN,NaN,NaN,manual_reg,,global
6,mean_sing_vlsfo_crack_trade_month_lag3,0.0,NaN,NaN,NaN,NaN,NaN,NaN,manual_reg,,global
7,new_sweet_sr_margin,0.0,NaN,NaN,NaN,NaN,NaN,NaN,manual_reg,,global
8,totaltar,0.8,NaN,NaN,NaN,NaN,NaN,NaN,manual_reg,,global


,metric,value,split,model,model_group_name,group
0,rmse,69.933254,train,manual_reg,,global
1,mae,49.314381,train,manual_reg,,global
2,mape,72.450355,train,manual_reg,,global
3,r_squared,-2.934418,train,manual_reg,,global
4,rmse,32.714562,test,manual_reg,,global
5,mae,25.8094,test,manual_reg,,global
6,mape,20.741383,test,manual_reg,,global
7,r_squared,-1.768217,test,manual_reg,,global
8,formula,target ~ mean_med_diesel_crack_input1_trade_mo...,,manual_reg,,global
9,model_type,manual_reg,,manual_reg,,global


In [33]:
spec_boost_xgb = boost_tree().set_engine('catboost')

fit_boost = spec_boost_xgb.fit(train_data,FORMULA_STR,original_training_data=train_data)
fit_boost = fit_boost.evaluate(test_data,original_test_data=test_data)

'fit_boost = spec_boost_xgb.fit(train_data,FORMULA_STR)'
'fit_boost = fit_boost.evaluate(test_data)'

outputs_boost, coefs_boost, stats_boost = fit_boost.extract_outputs()

display(outputs_boost)
display(coefs_boost)
display(stats_boost)

fig = plot_forecast(fit_boost, title="Sales Forecast")
fig.show()

,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,137.307014,137.65,0.342986,train,boost_tree,,global
1,2020-05-01,113.53,113.849423,113.53,-0.319423,train,boost_tree,,global
2,2020-06-01,43.31,43.639134,43.31,-0.329134,train,boost_tree,,global
3,2020-07-01,79.77,79.830402,79.77,-0.060402,train,boost_tree,,global
4,2020-08-01,48.27,48.450137,48.27,-0.180137,train,boost_tree,,global
5,2020-09-01,98.01,97.507512,98.01,0.502488,train,boost_tree,,global
6,2020-10-01,49.77,49.836597,49.77,-0.066597,train,boost_tree,,global
7,2020-11-01,35.48,35.831549,35.48,-0.351549,train,boost_tree,,global
8,2020-12-01,63.98,64.053103,63.98,-0.073103,train,boost_tree,,global
9,2021-01-01,109.76,109.396457,109.76,0.363543,train,boost_tree,,global


,variable,importance,model,model_group_name,group
0,new_sweet_sr_margin,17.654186,boost_tree,,global
1,mean_sing_gasoline_vs_vlsfo_trade_month,16.018210,boost_tree,,global
2,mean_nwe_hsfo_crack_trade_month_lag1,16.016766,boost_tree,,global
3,mean_med_diesel_crack_input1_trade_month_lag2,13.913826,boost_tree,,global
4,mean_sing_vlsfo_crack_trade_month_lag3,12.612833,boost_tree,,global
5,mean_nwe_ulsfo_crack_trade_month_lag3,12.298135,boost_tree,,global
6,mean_nwe_lsfo_crack_trade_month,8.175568,boost_tree,,global
7,totaltar,3.310477,boost_tree,,global
8,Intercept,0.000000,boost_tree,,global


,metric,value,split,model,model_group_name,group
0,rmse,0.296197,train,boost_tree,,global
1,mae,0.244692,train,boost_tree,,global
2,mape,0.282136,train,boost_tree,,global
3,smape,0.281809,train,boost_tree,,global
4,r_squared,0.999929,train,boost_tree,,global
5,mda,97.560976,train,boost_tree,,global
6,rmse,30.75166,test,boost_tree,,global
7,mae,24.552713,test,boost_tree,,global
8,mape,18.11122,test,boost_tree,,global
9,smape,20.677735,test,boost_tree,,global


## 1.5 - Decision Tree (Regression)

In [34]:
from py_parsnip import decision_tree

# Create specification - single decision tree
spec_decision_tree = decision_tree(
    tree_depth=10,
    min_n=5
).set_mode('regression')

# Fit model
fit_decision_tree = spec_decision_tree.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_decision_tree = fit_decision_tree.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs
outputs_decision_tree, coefs_decision_tree, stats_decision_tree = fit_decision_tree.extract_outputs()
display(outputs_decision_tree)
display(coefs_decision_tree)
display(stats_decision_tree)

fig = plot_forecast(fit_decision_tree, title="Sales Forecast")
fig.show()

Model fitted successfully!


,date,actuals,fitted,forecast,split,model,model_group_name,group,residuals
0,2020-04-01,137.65,137.650000,137.65,train,decision_tree,,global,137.650
1,2020-05-01,113.53,113.530000,113.53,train,decision_tree,,global,113.530
2,2020-06-01,43.31,68.340000,43.31,train,decision_tree,,global,43.310
3,2020-07-01,79.77,87.926667,79.77,train,decision_tree,,global,79.770
4,2020-08-01,48.27,49.375000,48.27,train,decision_tree,,global,48.270
5,2020-09-01,98.01,98.010000,98.01,train,decision_tree,,global,98.010
6,2020-10-01,49.77,49.375000,49.77,train,decision_tree,,global,49.770
7,2020-11-01,35.48,49.375000,35.48,train,decision_tree,,global,35.480
8,2020-12-01,63.98,49.375000,63.98,train,decision_tree,,global,63.980
9,2021-01-01,109.76,103.500000,109.76,train,decision_tree,,global,109.760


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,mean_med_diesel_crack_input1_trade_month_lag2,0.066205,NaN,NaN,NaN,NaN,NaN,NaN,decision_tree,,global
1,mean_nwe_hsfo_crack_trade_month_lag1,0.097251,NaN,NaN,NaN,NaN,NaN,NaN,decision_tree,,global
2,mean_nwe_lsfo_crack_trade_month,0.031971,NaN,NaN,NaN,NaN,NaN,NaN,decision_tree,,global
3,mean_nwe_ulsfo_crack_trade_month_lag3,0.217717,NaN,NaN,NaN,NaN,NaN,NaN,decision_tree,,global
4,mean_sing_gasoline_vs_vlsfo_trade_month,0.302798,NaN,NaN,NaN,NaN,NaN,NaN,decision_tree,,global
5,mean_sing_vlsfo_crack_trade_month_lag3,0.061646,NaN,NaN,NaN,NaN,NaN,NaN,decision_tree,,global
6,new_sweet_sr_margin,0.222412,NaN,NaN,NaN,NaN,NaN,NaN,decision_tree,,global
7,totaltar,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,decision_tree,,global


,metric,value,split,model,model_group_name,group
0,rmse,8.103318,train,decision_tree,,global
1,mae,5.249603,train,decision_tree,,global
2,mape,6.979143,train,decision_tree,,global
3,smape,6.651934,train,decision_tree,,global
4,r_squared,0.947175,train,decision_tree,,global
5,mda,95.121951,train,decision_tree,,global
6,adj_r_squared,0.934369,train,decision_tree,,global
7,rmse,36.490466,test,decision_tree,,global
8,mae,26.693333,test,decision_tree,,global
9,mape,19.818574,test,decision_tree,,global


## 1.6 - Random Forest

In [35]:
from py_parsnip import rand_forest

# Create specification - random forest
spec_rand_forest = rand_forest(
    trees=100,
    mtry=3,
    min_n=5
).set_mode('regression')

# Fit model
fit_rand_forest = spec_rand_forest.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_rand_forest = fit_rand_forest.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs
outputs_rand_forest, coefs_rand_forest, stats_rand_forest = fit_rand_forest.extract_outputs()
display(outputs_rand_forest)
display(coefs_rand_forest)
display(stats_rand_forest)

fig = plot_forecast(fit_rand_forest, title="Sales Forecast")
fig.show()

Model fitted successfully!


,date,actuals,fitted,forecast,split,model,model_group_name,group,residuals
0,2020-04-01,137.65,118.197379,137.65,train,rand_forest,,global,137.650000
1,2020-05-01,113.53,113.770795,113.53,train,rand_forest,,global,113.530000
2,2020-06-01,43.31,71.130278,43.31,train,rand_forest,,global,43.310000
3,2020-07-01,79.77,88.286091,79.77,train,rand_forest,,global,79.770000
4,2020-08-01,48.27,58.055551,48.27,train,rand_forest,,global,48.270000
5,2020-09-01,98.01,73.288958,98.01,train,rand_forest,,global,98.010000
6,2020-10-01,49.77,56.478575,49.77,train,rand_forest,,global,49.770000
7,2020-11-01,35.48,58.555302,35.48,train,rand_forest,,global,35.480000
8,2020-12-01,63.98,63.912617,63.98,train,rand_forest,,global,63.980000
9,2021-01-01,109.76,103.005424,109.76,train,rand_forest,,global,109.760000


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,mean_med_diesel_crack_input1_trade_month_lag2,0.076264,NaN,NaN,NaN,NaN,NaN,NaN,rand_forest,,global
1,mean_nwe_hsfo_crack_trade_month_lag1,0.170711,NaN,NaN,NaN,NaN,NaN,NaN,rand_forest,,global
2,mean_nwe_lsfo_crack_trade_month,0.073702,NaN,NaN,NaN,NaN,NaN,NaN,rand_forest,,global
3,mean_nwe_ulsfo_crack_trade_month_lag3,0.138578,NaN,NaN,NaN,NaN,NaN,NaN,rand_forest,,global
4,mean_sing_gasoline_vs_vlsfo_trade_month,0.205150,NaN,NaN,NaN,NaN,NaN,NaN,rand_forest,,global
5,mean_sing_vlsfo_crack_trade_month_lag3,0.139707,NaN,NaN,NaN,NaN,NaN,NaN,rand_forest,,global
6,new_sweet_sr_margin,0.149896,NaN,NaN,NaN,NaN,NaN,NaN,rand_forest,,global
7,totaltar,0.045992,NaN,NaN,NaN,NaN,NaN,NaN,rand_forest,,global


,metric,value,split,model,model_group_name,group
0,rmse,14.856048,train,rand_forest,,global
1,mae,11.291289,train,rand_forest,,global
2,mape,15.313078,train,rand_forest,,global
3,smape,13.538918,train,rand_forest,,global
4,r_squared,0.82245,train,rand_forest,,global
5,mda,78.04878,train,rand_forest,,global
6,adj_r_squared,0.779408,train,rand_forest,,global
7,rmse,31.304536,test,rand_forest,,global
8,mae,25.462717,test,rand_forest,,global
9,mape,18.796678,test,rand_forest,,global


## 1.7 - Gradient Boosting Models

### XGBoost Engine

In [36]:
from py_parsnip import boost_tree

# Create specification - XGBoost
spec_xgboost = boost_tree(
    trees=100,
    tree_depth=6,
    learn_rate=0.1
).set_engine('xgboost')

# Fit model
fit_xgboost = spec_xgboost.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_xgboost = fit_xgboost.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs
outputs_xgboost, coefs_xgboost, stats_xgboost = fit_xgboost.extract_outputs()
display(outputs_xgboost)
display(coefs_xgboost)
display(stats_xgboost)

fig = plot_forecast(fit_xgboost, title="Sales Forecast")
fig.show()

Model fitted successfully!


,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,137.371353,137.65,0.278647,train,boost_tree,,global
1,2020-05-01,113.53,113.619263,113.53,-0.089263,train,boost_tree,,global
2,2020-06-01,43.31,43.780792,43.31,-0.470792,train,boost_tree,,global
3,2020-07-01,79.77,80.011681,79.77,-0.241681,train,boost_tree,,global
4,2020-08-01,48.27,48.409065,48.27,-0.139065,train,boost_tree,,global
5,2020-09-01,98.01,97.717262,98.01,0.292738,train,boost_tree,,global
6,2020-10-01,49.77,49.850319,49.77,-0.080319,train,boost_tree,,global
7,2020-11-01,35.48,35.899136,35.48,-0.419136,train,boost_tree,,global
8,2020-12-01,63.98,64.026955,63.98,-0.046955,train,boost_tree,,global
9,2021-01-01,109.76,109.648453,109.76,0.111547,train,boost_tree,,global


,variable,importance,model,model_group_name,group
0,mean_nwe_ulsfo_crack_trade_month_lag3,0.321489,boost_tree,,global
1,new_sweet_sr_margin,0.268446,boost_tree,,global
2,mean_sing_gasoline_vs_vlsfo_trade_month,0.213121,boost_tree,,global
3,mean_nwe_hsfo_crack_trade_month_lag1,0.104401,boost_tree,,global
4,mean_sing_vlsfo_crack_trade_month_lag3,0.040838,boost_tree,,global
5,mean_med_diesel_crack_input1_trade_month_lag2,0.023977,boost_tree,,global
6,mean_nwe_lsfo_crack_trade_month,0.019344,boost_tree,,global
7,totaltar,0.008384,boost_tree,,global
8,Intercept,0.000000,boost_tree,,global


,metric,value,split,model,model_group_name,group
0,rmse,0.214397,train,boost_tree,,global
1,mae,0.167988,train,boost_tree,,global
2,mape,0.21335,train,boost_tree,,global
3,smape,0.212874,train,boost_tree,,global
4,r_squared,0.999963,train,boost_tree,,global
5,mda,100.0,train,boost_tree,,global
6,rmse,36.025231,test,boost_tree,,global
7,mae,28.110579,test,boost_tree,,global
8,mape,20.783101,test,boost_tree,,global
9,smape,24.652443,test,boost_tree,,global


### CatBoost Engine

In [37]:
# Create specification - CatBoost
spec_catboost = boost_tree(
    trees=100,
    tree_depth=6,
    learn_rate=0.1
).set_engine('catboost')

# Fit model
fit_catboost = spec_catboost.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_catboost = fit_catboost.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs
outputs_catboost, coefs_catboost, stats_catboost = fit_catboost.extract_outputs()
display(outputs_catboost)
display(coefs_catboost)
display(stats_catboost)

fig = plot_forecast(fit_catboost, title="Sales Forecast")
fig.show()

Model fitted successfully!


,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,129.939408,137.65,7.710592,train,boost_tree,,global
1,2020-05-01,113.53,115.791573,113.53,-2.261573,train,boost_tree,,global
2,2020-06-01,43.31,53.027737,43.31,-9.717737,train,boost_tree,,global
3,2020-07-01,79.77,84.781239,79.77,-5.011239,train,boost_tree,,global
4,2020-08-01,48.27,53.328801,48.27,-5.058801,train,boost_tree,,global
5,2020-09-01,98.01,90.127422,98.01,7.882578,train,boost_tree,,global
6,2020-10-01,49.77,53.323350,49.77,-3.553350,train,boost_tree,,global
7,2020-11-01,35.48,43.769468,35.48,-8.289468,train,boost_tree,,global
8,2020-12-01,63.98,67.419298,63.98,-3.439298,train,boost_tree,,global
9,2021-01-01,109.76,104.628413,109.76,5.131587,train,boost_tree,,global


,variable,importance,model,model_group_name,group
0,mean_nwe_ulsfo_crack_trade_month_lag3,20.130475,boost_tree,,global
1,new_sweet_sr_margin,18.946928,boost_tree,,global
2,mean_sing_gasoline_vs_vlsfo_trade_month,13.470332,boost_tree,,global
3,mean_med_diesel_crack_input1_trade_month_lag2,12.711272,boost_tree,,global
4,mean_nwe_hsfo_crack_trade_month_lag1,12.706198,boost_tree,,global
5,mean_sing_vlsfo_crack_trade_month_lag3,11.643147,boost_tree,,global
6,mean_nwe_lsfo_crack_trade_month,6.981899,boost_tree,,global
7,totaltar,3.409748,boost_tree,,global
8,Intercept,0.000000,boost_tree,,global


,metric,value,split,model,model_group_name,group
0,rmse,5.144185,train,boost_tree,,global
1,mae,4.570594,train,boost_tree,,global
2,mape,5.770055,train,boost_tree,,global
3,smape,5.577422,train,boost_tree,,global
4,r_squared,0.978711,train,boost_tree,,global
5,mda,97.560976,train,boost_tree,,global
6,rmse,29.429382,test,boost_tree,,global
7,mae,22.358302,test,boost_tree,,global
8,mape,16.26982,test,boost_tree,,global
9,smape,18.491759,test,boost_tree,,global


## 1.8 - Support Vector Machines

### SVM with RBF Kernel

In [38]:
from py_parsnip import svm_rbf

# Create specification - SVM with RBF kernel
spec_svm_rbf = svm_rbf(
    cost=1.0,
    rbf_sigma=0.1
).set_mode('regression')

# Fit model
fit_svm_rbf = spec_svm_rbf.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_svm_rbf = fit_svm_rbf.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs
outputs_svm_rbf, coefs_svm_rbf, stats_svm_rbf = fit_svm_rbf.extract_outputs()
display(outputs_svm_rbf)
display(coefs_svm_rbf)
display(stats_svm_rbf)

fig = plot_forecast(fit_svm_rbf, title="Sales Forecast")
fig.show()

Model fitted successfully!


,date,actuals,fitted,forecast,split,model,model_group_name,group,residuals
0,2020-04-01,137.65,100.479810,137.65,train,svm_rbf,,global,137.650000
1,2020-05-01,113.53,100.479810,113.53,train,svm_rbf,,global,113.530000
2,2020-06-01,43.31,98.479661,43.31,train,svm_rbf,,global,43.310000
3,2020-07-01,79.77,98.479660,79.77,train,svm_rbf,,global,79.770000
4,2020-08-01,48.27,98.368319,48.27,train,svm_rbf,,global,48.270000
5,2020-09-01,98.01,98.109693,98.01,train,svm_rbf,,global,98.010000
6,2020-10-01,49.77,97.990034,49.77,train,svm_rbf,,global,49.770000
7,2020-11-01,35.48,98.282578,35.48,train,svm_rbf,,global,35.480000
8,2020-12-01,63.98,98.415650,63.98,train,svm_rbf,,global,63.980000
9,2021-01-01,109.76,99.937587,109.76,train,svm_rbf,,global,109.760000


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group


,metric,value,split,model,model_group_name,group
0,rmse,34.482278,train,svm_rbf,,global
1,mae,26.03483,train,svm_rbf,,global
2,mape,37.464894,train,svm_rbf,,global
3,smape,28.30255,train,svm_rbf,,global
4,r_squared,0.043456,train,svm_rbf,,global
5,mda,70.731707,train,svm_rbf,,global
6,adj_r_squared,-0.188433,train,svm_rbf,,global
7,rmse,32.21439,test,svm_rbf,,global
8,mae,25.517632,test,svm_rbf,,global
9,mape,18.640692,test,svm_rbf,,global


### SVM with Linear Kernel

In [39]:
from py_parsnip import svm_linear

# Create specification - SVM with linear kernel
spec_svm_linear = svm_linear(
    cost=1.0
).set_mode('regression')

# Fit model
fit_svm_linear = spec_svm_linear.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_svm_linear = fit_svm_linear.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs
outputs_svm_linear, coefs_svm_linear, stats_svm_linear = fit_svm_linear.extract_outputs()
display(outputs_svm_linear)
display(coefs_svm_linear)
display(stats_svm_linear)

fig = plot_forecast(fit_svm_linear, title="Sales Forecast")
fig.show()

Model fitted successfully!


/Users/matthewdeane/Documents/Data Science/python/_projects/py-tidymodels/py-tidymodels2/lib/python3.10/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.



,date,actuals,fitted,forecast,split,model,model_group_name,group,residuals
0,2020-04-01,137.65,135.771912,137.65,train,svm_linear,,global,137.650000
1,2020-05-01,113.53,110.097955,113.53,train,svm_linear,,global,113.530000
2,2020-06-01,43.31,43.310000,43.31,train,svm_linear,,global,43.310000
3,2020-07-01,79.77,66.366490,79.77,train,svm_linear,,global,79.770000
4,2020-08-01,48.27,48.053298,48.27,train,svm_linear,,global,48.270000
5,2020-09-01,98.01,60.667935,98.01,train,svm_linear,,global,98.010000
6,2020-10-01,49.77,69.421192,49.77,train,svm_linear,,global,49.770000
7,2020-11-01,35.48,64.736511,35.48,train,svm_linear,,global,35.480000
8,2020-12-01,63.98,47.263237,63.98,train,svm_linear,,global,63.980000
9,2021-01-01,109.76,56.279195,109.76,train,svm_linear,,global,109.760000


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,mean_med_diesel_crack_input1_trade_month_lag2,-0.333609,NaN,NaN,NaN,NaN,NaN,NaN,svm_linear,,global
1,mean_nwe_hsfo_crack_trade_month_lag1,-0.697217,NaN,NaN,NaN,NaN,NaN,NaN,svm_linear,,global
2,mean_nwe_lsfo_crack_trade_month,0.396663,NaN,NaN,NaN,NaN,NaN,NaN,svm_linear,,global
3,mean_nwe_ulsfo_crack_trade_month_lag3,4.576133,NaN,NaN,NaN,NaN,NaN,NaN,svm_linear,,global
4,mean_sing_gasoline_vs_vlsfo_trade_month,-0.117020,NaN,NaN,NaN,NaN,NaN,NaN,svm_linear,,global
5,mean_sing_vlsfo_crack_trade_month_lag3,-0.948369,NaN,NaN,NaN,NaN,NaN,NaN,svm_linear,,global
6,new_sweet_sr_margin,-4.192003,NaN,NaN,NaN,NaN,NaN,NaN,svm_linear,,global
7,totaltar,-0.770948,NaN,NaN,NaN,NaN,NaN,NaN,svm_linear,,global


,metric,value,split,model,model_group_name,group
0,rmse,41.850311,train,svm_linear,,global
1,mae,29.053743,train,svm_linear,,global
2,mape,27.42038,train,svm_linear,,global
3,smape,31.105907,train,svm_linear,,global
4,r_squared,-0.408998,train,svm_linear,,global
5,mda,63.414634,train,svm_linear,,global
6,adj_r_squared,-0.750573,train,svm_linear,,global
7,rmse,72.684548,test,svm_linear,,global
8,mae,65.935208,test,svm_linear,,global
9,mape,51.113949,test,svm_linear,,global


## 1.9 - k-Nearest Neighbors

In [40]:
from py_parsnip import nearest_neighbor

# Create specification - k-NN with k=5
spec_knn_5 = nearest_neighbor(
    neighbors=5,
    weight_func='uniform'
).set_mode('regression')

# Fit model
fit_knn_5 = spec_knn_5.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_knn_5 = fit_knn_5.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs
outputs_knn_5, coefs_knn_5, stats_knn_5 = fit_knn_5.extract_outputs()
display(outputs_knn_5)
display(coefs_knn_5)
display(stats_knn_5)

fig = plot_forecast(fit_knn_5, title="Sales Forecast")
fig.show()

Model fitted successfully!


,date,actuals,fitted,forecast,split,model,model_group_name,group,residuals
0,2020-04-01,137.65,124.480,137.65,train,nearest_neighbor,,global,137.650
1,2020-05-01,113.53,105.616,113.53,train,nearest_neighbor,,global,113.530
2,2020-06-01,43.31,75.824,43.31,train,nearest_neighbor,,global,43.310
3,2020-07-01,79.77,75.824,79.77,train,nearest_neighbor,,global,79.770
4,2020-08-01,48.27,73.958,48.27,train,nearest_neighbor,,global,48.270
5,2020-09-01,98.01,68.258,98.01,train,nearest_neighbor,,global,98.010
6,2020-10-01,49.77,71.400,49.77,train,nearest_neighbor,,global,49.770
7,2020-11-01,35.48,71.400,35.48,train,nearest_neighbor,,global,35.480
8,2020-12-01,63.98,71.400,63.98,train,nearest_neighbor,,global,63.980
9,2021-01-01,109.76,71.400,109.76,train,nearest_neighbor,,global,109.760


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group


,metric,value,split,model,model_group_name,group
0,rmse,28.158769,train,nearest_neighbor,,global
1,mae,22.331476,train,nearest_neighbor,,global
2,mape,29.769022,train,nearest_neighbor,,global
3,smape,24.847481,train,nearest_neighbor,,global
4,r_squared,0.362118,train,nearest_neighbor,,global
5,mda,51.219512,train,nearest_neighbor,,global
6,adj_r_squared,0.20748,train,nearest_neighbor,,global
7,rmse,30.03854,test,nearest_neighbor,,global
8,mae,22.409867,test,nearest_neighbor,,global
9,mape,16.338002,test,nearest_neighbor,,global


In [41]:
# k-NN with k=10 and distance weighting
spec_knn_10 = nearest_neighbor(
    neighbors=10,
    weight_func='distance'
).set_mode('regression')

# Fit model
fit_knn_10 = spec_knn_10.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_knn_10 = fit_knn_10.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs
outputs_knn_10, coefs_knn_10, stats_knn_10 = fit_knn_10.extract_outputs()
display(outputs_knn_10)
display(coefs_knn_10)
display(stats_knn_10)

fig = plot_forecast(fit_knn_10, title="Sales Forecast")
fig.show()

Model fitted successfully!


,date,actuals,fitted,forecast,split,model,model_group_name,group,residuals
0,2020-04-01,137.65,137.650000,137.65,train,nearest_neighbor,,global,137.650000
1,2020-05-01,113.53,113.530000,113.53,train,nearest_neighbor,,global,113.530000
2,2020-06-01,43.31,43.310000,43.31,train,nearest_neighbor,,global,43.310000
3,2020-07-01,79.77,79.770000,79.77,train,nearest_neighbor,,global,79.770000
4,2020-08-01,48.27,48.270000,48.27,train,nearest_neighbor,,global,48.270000
5,2020-09-01,98.01,98.010000,98.01,train,nearest_neighbor,,global,98.010000
6,2020-10-01,49.77,49.770000,49.77,train,nearest_neighbor,,global,49.770000
7,2020-11-01,35.48,35.480000,35.48,train,nearest_neighbor,,global,35.480000
8,2020-12-01,63.98,63.980000,63.98,train,nearest_neighbor,,global,63.980000
9,2021-01-01,109.76,109.760000,109.76,train,nearest_neighbor,,global,109.760000


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group


,metric,value,split,model,model_group_name,group
0,rmse,0.0,train,nearest_neighbor,,global
1,mae,0.0,train,nearest_neighbor,,global
2,mape,0.0,train,nearest_neighbor,,global
3,smape,0.0,train,nearest_neighbor,,global
4,r_squared,1.0,train,nearest_neighbor,,global
5,mda,100.0,train,nearest_neighbor,,global
6,adj_r_squared,1.0,train,nearest_neighbor,,global
7,rmse,26.477439,test,nearest_neighbor,,global
8,mae,19.463516,test,nearest_neighbor,,global
9,mape,13.969614,test,nearest_neighbor,,global


## 1.10 - Multi-Layer Perceptron (Neural Network)

In [42]:
from py_parsnip import mlp

# Create specification - MLP with single hidden layer
spec_mlp = mlp(
    hidden_units=50,
    epochs=100,
    learn_rate=0.01
).set_mode('regression')

# Fit model
fit_mlp = spec_mlp.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_mlp = fit_mlp.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs
outputs_mlp, coefs_mlp, stats_mlp = fit_mlp.extract_outputs()
display(outputs_mlp)
display(coefs_mlp)
display(stats_mlp)

fig = plot_forecast(fit_mlp, title="Sales Forecast")
fig.show()

Model fitted successfully!


,date,actuals,fitted,forecast,split,model,model_group_name,group,residuals
0,2020-04-01,137.65,121.431596,137.65,train,mlp,,global,137.650000
1,2020-05-01,113.53,88.054614,113.53,train,mlp,,global,113.530000
2,2020-06-01,43.31,58.783771,43.31,train,mlp,,global,43.310000
3,2020-07-01,79.77,57.468154,79.77,train,mlp,,global,79.770000
4,2020-08-01,48.27,63.942430,48.27,train,mlp,,global,48.270000
5,2020-09-01,98.01,67.861736,98.01,train,mlp,,global,98.010000
6,2020-10-01,49.77,69.969965,49.77,train,mlp,,global,49.770000
7,2020-11-01,35.48,68.984918,35.48,train,mlp,,global,35.480000
8,2020-12-01,63.98,63.239941,63.98,train,mlp,,global,63.980000
9,2021-01-01,109.76,67.666797,109.76,train,mlp,,global,109.760000


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,layer_0_to_1,-0.010764,0.217971,NaN,NaN,-0.388685,0.408987,NaN,mlp,,global
1,layer_1_to_2,0.116793,0.237155,NaN,NaN,-0.281151,0.456851,NaN,mlp,,global


,metric,value,split,model,model_group_name,group
0,rmse,37.956444,train,mlp,,global
1,mae,31.794481,train,mlp,,global
2,mape,37.684813,train,mlp,,global
3,smape,34.62944,train,mlp,,global
4,r_squared,-0.159002,train,mlp,,global
5,mda,56.097561,train,mlp,,global
6,adj_r_squared,-0.439972,train,mlp,,global
7,rmse,42.082041,test,mlp,,global
8,mae,35.141776,test,mlp,,global
9,mape,26.77284,test,mlp,,global


## 1.11 - Poisson Regression (Count Data)

In [43]:
from py_parsnip import poisson_reg

# Create specification - Poisson regression
spec_poisson = poisson_reg()

# Fit model
fit_poisson = spec_poisson.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_poisson = fit_poisson.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs
outputs_poisson, coefs_poisson, stats_poisson = fit_poisson.extract_outputs()
display(outputs_poisson)
display(coefs_poisson)
display(stats_poisson)

fig = plot_forecast(fit_poisson, title="Sales Forecast")
fig.show()

Model fitted successfully!


/Users/matthewdeane/Documents/Data Science/python/_projects/py-tidymodels/py-tidymodels2/lib/python3.10/site-packages/statsmodels/genmod/generalized_linear_model.py:1923: FutureWarning:

The bic value is computed using the deviance formula. After 0.13 this will change to the log-likelihood based formula. This change has no impact on the relative rank of models compared using BIC. You can directly access the log-likelihood version using the `bic_llf` attribute. You can suppress this message by calling statsmodels.genmod.generalized_linear_model.SET_USE_BIC_LLF with True to get the LLF-based version now or False to retainthe deviance version.



,date,actuals,fitted,forecast,residuals,split,pearson_resid,deviance_resid,model,model_group_name,group
0,2020-04-01,137.65,100.119538,137.65,37.530462,train,3.750805,3.546923,poisson_reg,,global
1,2020-05-01,113.53,98.316324,113.53,15.213676,train,1.534339,1.497120,poisson_reg,,global
2,2020-06-01,43.31,100.950237,43.31,-57.640237,train,-5.736831,-6.479105,poisson_reg,,global
3,2020-07-01,79.77,95.638292,79.77,-15.868292,train,-1.622610,-1.670902,poisson_reg,,global
4,2020-08-01,48.27,89.364110,48.27,-41.094110,train,-4.347084,-4.767429,poisson_reg,,global
5,2020-09-01,98.01,92.852840,98.01,5.157160,train,0.535196,0.530353,poisson_reg,,global
6,2020-10-01,49.77,89.830613,49.77,-40.060613,train,-4.226739,-4.619688,poisson_reg,,global
7,2020-11-01,35.48,86.243375,35.48,-50.763375,train,-5.466226,-6.204820,poisson_reg,,global
8,2020-12-01,63.98,85.788475,63.98,-21.808475,train,-2.354566,-2.466672,poisson_reg,,global
9,2021-01-01,109.76,88.354928,109.76,21.405072,train,2.277201,2.193456,poisson_reg,,global


,variable,coefficient,std_error,z_stat,p_value,ci_0.025,ci_0.975,model,model_group_name,group
0,Intercept,4.463937,0.052978,84.260245,0.000000e+00,4.360102,4.567772,poisson_reg,,global
1,mean_med_diesel_crack_input1_trade_month_lag2,-0.000538,0.000339,-1.587425,1.124163e-01,-0.001203,0.000126,poisson_reg,,global
2,mean_nwe_hsfo_crack_trade_month_lag1,-0.003221,0.001493,-2.156845,3.101778e-02,-0.006147,-0.000294,poisson_reg,,global
3,mean_nwe_lsfo_crack_trade_month,-0.005751,0.003292,-1.746711,8.068751e-02,-0.012204,0.000702,poisson_reg,,global
4,mean_nwe_ulsfo_crack_trade_month_lag3,0.003486,0.007604,0.458455,6.466253e-01,-0.011417,0.018390,poisson_reg,,global
5,mean_sing_gasoline_vs_vlsfo_trade_month,0.013514,0.003703,3.649082,2.631789e-04,0.006255,0.020772,poisson_reg,,global
6,mean_sing_vlsfo_crack_trade_month_lag3,0.006457,0.009679,0.667082,5.047200e-01,-0.012513,0.025427,poisson_reg,,global
7,new_sweet_sr_margin,0.003774,0.006349,0.594340,5.522844e-01,-0.008671,0.016218,poisson_reg,,global
8,totaltar,-0.005095,0.000731,-6.967101,3.235379e-12,-0.006529,-0.003662,poisson_reg,,global


,metric,value,split,model,model_group_name,group
0,rmse,30.752869,train,poisson_reg,,global
1,mae,25.120068,train,poisson_reg,,global
2,mape,30.670331,train,poisson_reg,,global
3,poisson_deviance,9.669116,train,poisson_reg,,global
4,pseudo_r_squared,0.270977,train,poisson_reg,,global
5,rmse,45.633504,test,poisson_reg,,global
6,mae,37.394905,test,poisson_reg,,global
7,mape,28.214819,test,poisson_reg,,global
8,poisson_deviance,21.419296,test,poisson_reg,,global
9,pseudo_r_squared,-6.236824,test,poisson_reg,,global


## 1.12 - MARS (Multivariate Adaptive Regression Splines)

In [44]:
# # ! pip install pyearth
# import pyearth
# from py_parsnip import mars

# # Create specification - MARS
# spec_mars = mars(
#     num_terms=10,
#     prod_degree=2
# )

# # Fit model
# fit_mars = spec_mars.fit(train_data, FORMULA_STR)

# # Evaluate on test set
# fit_mars = fit_mars.evaluate(test_data)

# print("Model fitted successfully!")

# # Extract outputs
# outputs_mars, coefs_mars, stats_mars = fit_mars.extract_outputs()
# display(outputs_mars)
# display(coefs_mars)
# display(stats_mars)

## 1.13 - Seasonal Decomposition (STL)

In [45]:
from py_parsnip import seasonal_reg

# Create specification - STL decomposition with monthly seasonality
spec_stl = seasonal_reg(
    seasonal_period_1=12  # Monthly seasonality
)

# Fit model
fit_stl = spec_stl.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_stl = fit_stl.evaluate(test_data)

print("STL model fitted successfully!")

# Extract outputs
outputs_stl, coefs_stl, stats_stl = fit_stl.extract_outputs()
display(outputs_stl)
display(coefs_stl)
display(stats_stl)

fig = plot_forecast(fit_stl, title="Sales Forecast")
fig.show()

STL model fitted successfully!


,date,actuals,fitted,forecast,residuals,trend,seasonal,remainder,split,model,model_group_name,group
0,2020-04-01,137.65,117.619315,137.65,20.030685,70.143758,36.620755,30.885487,train,seasonal_reg,,global
1,2020-05-01,113.53,133.402886,113.53,-19.872886,71.910306,43.494343,-1.874649,train,seasonal_reg,,global
2,2020-06-01,43.31,72.183997,43.31,-28.873997,73.785064,-10.693467,-19.781597,train,seasonal_reg,,global
3,2020-07-01,79.77,62.429794,79.77,17.340206,75.749220,-9.820741,13.841521,train,seasonal_reg,,global
4,2020-08-01,48.27,55.256535,48.27,-6.986535,77.792935,-24.829164,-4.693771,train,seasonal_reg,,global
5,2020-09-01,98.01,101.662938,98.01,-3.652938,79.900334,23.460354,-5.350688,train,seasonal_reg,,global
6,2020-10-01,49.77,61.087768,49.77,-11.317768,82.055107,-16.563438,-15.721670,train,seasonal_reg,,global
7,2020-11-01,35.48,33.788594,35.48,1.691406,84.247219,-40.249214,-8.518005,train,seasonal_reg,,global
8,2020-12-01,63.98,73.364252,63.98,-9.384252,86.488525,-2.257188,-20.251338,train,seasonal_reg,,global
9,2021-01-01,109.76,89.725278,109.76,20.034722,88.870571,16.944817,3.944612,train,seasonal_reg,,global


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,alpha (smoothing_level),0.399490,NaN,NaN,NaN,NaN,NaN,NaN,seasonal_reg,,global
1,beta (smoothing_trend),0.000000,NaN,NaN,NaN,NaN,NaN,NaN,seasonal_reg,,global
2,strength_seasonality,0.355614,NaN,NaN,NaN,NaN,NaN,NaN,seasonal_reg,,global


,metric,value,split,model,model_group_name,group
0,rmse,22.831837,train,seasonal_reg,,global
1,mae,19.047781,train,seasonal_reg,,global
2,mape,22.868679,train,seasonal_reg,,global
3,smape,20.830307,train,seasonal_reg,,global
4,r_squared,0.580633,train,seasonal_reg,,global
5,mda,73.170732,train,seasonal_reg,,global
6,rmse,38.267984,test,seasonal_reg,,global
7,mae,32.42551,test,seasonal_reg,,global
8,mape,25.763965,test,seasonal_reg,,global
9,smape,26.606802,test,seasonal_reg,,global


## 1.14 - ARIMA Variations

### Non-Seasonal ARIMA

In [46]:
from py_parsnip import arima_reg

# Create specification - Non-seasonal ARIMA(1,1,1)
spec_arima_nonseasonal = arima_reg(
    non_seasonal_ar=1,
    non_seasonal_differences=1,
    non_seasonal_ma=1
)

# Fit model
fit_arima_nonseasonal = spec_arima_nonseasonal.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_arima_nonseasonal = fit_arima_nonseasonal.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs
outputs_arima_ns, coefs_arima_ns, stats_arima_ns = fit_arima_nonseasonal.extract_outputs()
display(outputs_arima_ns)
display(coefs_arima_ns)
display(stats_arima_ns)

fig = plot_forecast(fit_arima_nonseasonal, title="Sales Forecast")
fig.show()

Model fitted successfully!


/Users/matthewdeane/Documents/Data Science/python/_projects/py-tidymodels/py-tidymodels2/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

/Users/matthewdeane/Documents/Data Science/python/_projects/py-tidymodels/py-tidymodels2/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.



,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,-84.442081,137.65,222.092081,train,arima_reg,,global
1,2020-05-01,113.53,170.298357,113.53,-56.768357,train,arima_reg,,global
2,2020-06-01,43.31,162.584576,43.31,-119.274576,train,arima_reg,,global
3,2020-07-01,79.77,71.698577,79.77,8.071423,train,arima_reg,,global
4,2020-08-01,48.27,72.131673,48.27,-23.861673,train,arima_reg,,global
5,2020-09-01,98.01,71.749209,98.01,26.260791,train,arima_reg,,global
6,2020-10-01,49.77,76.939793,49.77,-27.169793,train,arima_reg,,global
7,2020-11-01,35.48,58.462168,35.48,-22.982168,train,arima_reg,,global
8,2020-12-01,63.98,62.122412,63.98,1.857588,train,arima_reg,,global
9,2021-01-01,109.76,59.261462,109.76,50.498538,train,arima_reg,,global


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,mean_med_diesel_crack_input1_trade_month_lag2,0.041736,0.249243,0.167451,0.867015,-0.446772,0.530244,NaN,arima_reg,,global
1,mean_nwe_hsfo_crack_trade_month_lag1,0.105841,0.534546,0.198002,0.843044,-0.941851,1.153533,NaN,arima_reg,,global
2,mean_nwe_lsfo_crack_trade_month,-0.699411,2.179392,-0.320921,0.748271,-4.970941,3.572118,NaN,arima_reg,,global
3,mean_nwe_ulsfo_crack_trade_month_lag3,1.271434,3.276418,0.388056,0.697975,-5.150228,7.693095,NaN,arima_reg,,global
4,mean_sing_gasoline_vs_vlsfo_trade_month,1.540315,2.429186,0.634087,0.526024,-3.220802,6.301431,NaN,arima_reg,,global
5,mean_sing_vlsfo_crack_trade_month_lag3,-3.572266,4.122778,-0.866471,0.386232,-11.652763,4.508230,NaN,arima_reg,,global
6,new_sweet_sr_margin,3.278743,2.066676,1.586482,0.112630,-0.771867,7.329354,NaN,arima_reg,,global
7,totaltar,0.006771,0.621663,0.010892,0.991310,-1.211665,1.225207,NaN,arima_reg,,global
8,ar.L1,0.083033,0.287457,0.288852,0.772694,-0.480373,0.646439,NaN,arima_reg,,global
9,ma.L1,-0.757910,0.214939,-3.526170,0.000422,-1.179183,-0.336638,NaN,arima_reg,,global


,metric,value,split,model,model_group_name,group
0,rmse,48.375554,train,arima_reg,,global
1,mae,30.429038,train,arima_reg,,global
2,mape,34.311509,train,arima_reg,,global
3,smape,30.038352,train,arima_reg,,global
4,r_squared,-0.88263,train,arima_reg,,global
5,mda,41.463415,train,arima_reg,,global
6,rmse,44.070909,test,arima_reg,,global
7,mae,38.724819,test,arima_reg,,global
8,mape,29.69739,test,arima_reg,,global
9,smape,36.195736,test,arima_reg,,global


### Auto ARIMA (Automatic Order Selection)

In [47]:
# # Create specification - Auto ARIMA with search constraints
# spec_auto_arima = arima_reg(
#     non_seasonal_ar=5,      # max_p = 5
#     non_seasonal_differences=2,  # max_d = 2
#     non_seasonal_ma=5,      # max_q = 5
#     seasonal_period=12
# ).set_engine('auto_arima')

# # Fit model - will search for best (p,d,q) within constraints
# fit_auto_arima = spec_auto_arima.fit(train_data, FORMULA_STR)

# # Evaluate on test set
# fit_auto_arima = fit_auto_arima.evaluate(test_data)

# print("Model fitted successfully!")
# print(f"Selected order: {fit_auto_arima.fit_data.get('order', 'N/A')}")

# # Extract outputs
# outputs_auto_arima, coefs_auto_arima, stats_auto_arima = fit_auto_arima.extract_outputs()
# display(outputs_auto_arima)
# display(coefs_auto_arima)
# display(stats_auto_arima)

## 1.15 - Exponential Smoothing Variations

### Additive Seasonality

In [48]:
from py_parsnip import exp_smoothing

# ETS(A,A,A) - All additive
spec_ets_aaa = exp_smoothing(
    error='add',
    trend='add',
    season='add',
    seasonal_period=12
)

# Fit model
fit_ets_aaa = spec_ets_aaa.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_ets_aaa = fit_ets_aaa.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs
outputs_ets_aaa, coefs_ets_aaa, stats_ets_aaa = fit_ets_aaa.extract_outputs()
display(outputs_ets_aaa)
display(coefs_ets_aaa)
display(stats_ets_aaa)

fig = plot_forecast(fit_ets_aaa, title="Sales Forecast")
fig.show()

Model fitted successfully!


/Users/matthewdeane/Documents/Data Science/python/_projects/py-tidymodels/py-tidymodels2/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.



,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,119.568266,137.65,18.081734,train,exp_smoothing,,global
1,2020-05-01,113.53,102.034604,113.53,11.495396,train,exp_smoothing,,global
2,2020-06-01,43.31,72.949596,43.31,-29.639596,train,exp_smoothing,,global
3,2020-07-01,79.77,90.679617,79.77,-10.909617,train,exp_smoothing,,global
4,2020-08-01,48.27,78.521668,48.27,-30.251668,train,exp_smoothing,,global
5,2020-09-01,98.01,69.405629,98.01,28.604371,train,exp_smoothing,,global
6,2020-10-01,49.77,61.577041,49.77,-11.807041,train,exp_smoothing,,global
7,2020-11-01,35.48,89.866576,35.48,-54.386576,train,exp_smoothing,,global
8,2020-12-01,63.98,112.668013,63.98,-48.688013,train,exp_smoothing,,global
9,2021-01-01,109.76,94.222961,109.76,15.537039,train,exp_smoothing,,global


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,alpha (smoothing_level),1.490116e-08,NaN,NaN,NaN,NaN,NaN,NaN,exp_smoothing,,global
1,beta (smoothing_trend),0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,exp_smoothing,,global
2,gamma (smoothing_seasonal),0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,exp_smoothing,,global
3,phi (damping_trend),NaN,NaN,NaN,NaN,NaN,NaN,NaN,exp_smoothing,,global
4,l0 (initial_level),8.749568e+01,NaN,NaN,NaN,NaN,NaN,NaN,exp_smoothing,,global
5,b0 (initial_trend),4.893895e-01,NaN,NaN,NaN,NaN,NaN,NaN,exp_smoothing,,global


,metric,value,split,model,model_group_name,group
0,rmse,29.174087,train,exp_smoothing,,global
1,mae,24.100522,train,exp_smoothing,,global
2,mape,32.149773,train,exp_smoothing,,global
3,smape,26.919124,train,exp_smoothing,,global
4,r_squared,0.315289,train,exp_smoothing,,global
5,mda,68.292683,train,exp_smoothing,,global
6,rmse,26.522722,test,exp_smoothing,,global
7,mae,20.040026,test,exp_smoothing,,global
8,mape,15.517037,test,exp_smoothing,,global
9,smape,17.038668,test,exp_smoothing,,global


### Multiplicative Seasonality

In [49]:
# ETS(A,A,M) - Multiplicative seasonality
spec_ets_aam = exp_smoothing(
    error='add',
    trend='add',
    season='mul',
    seasonal_period=12
)

# Fit model
fit_ets_aam = spec_ets_aam.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_ets_aam = fit_ets_aam.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs
outputs_ets_aam, coefs_ets_aam, stats_ets_aam = fit_ets_aam.extract_outputs()
display(outputs_ets_aam)
display(coefs_ets_aam)
display(stats_ets_aam)

fig = plot_forecast(fit_ets_aam, title="Sales Forecast")
fig.show()

/Users/matthewdeane/Documents/Data Science/python/_projects/py-tidymodels/py-tidymodels2/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.



Model fitted successfully!


,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,114.332744,137.65,23.317256,train,exp_smoothing,,global
1,2020-05-01,113.53,97.365827,113.53,16.164173,train,exp_smoothing,,global
2,2020-06-01,43.31,72.870232,43.31,-29.560232,train,exp_smoothing,,global
3,2020-07-01,79.77,89.522937,79.77,-9.752937,train,exp_smoothing,,global
4,2020-08-01,48.27,78.817592,48.27,-30.547592,train,exp_smoothing,,global
5,2020-09-01,98.01,67.767290,98.01,30.242710,train,exp_smoothing,,global
6,2020-10-01,49.77,62.000122,49.77,-12.230122,train,exp_smoothing,,global
7,2020-11-01,35.48,90.453272,35.48,-54.973272,train,exp_smoothing,,global
8,2020-12-01,63.98,110.830834,63.98,-46.850834,train,exp_smoothing,,global
9,2021-01-01,109.76,92.116482,109.76,17.643518,train,exp_smoothing,,global


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,alpha (smoothing_level),1.490116e-08,NaN,NaN,NaN,NaN,NaN,NaN,exp_smoothing,,global
1,beta (smoothing_trend),0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,exp_smoothing,,global
2,gamma (smoothing_seasonal),0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,exp_smoothing,,global
3,phi (damping_trend),NaN,NaN,NaN,NaN,NaN,NaN,NaN,exp_smoothing,,global
4,l0 (initial_level),7.300620e+01,NaN,NaN,NaN,NaN,NaN,NaN,exp_smoothing,,global
5,b0 (initial_trend),5.023695e-01,NaN,NaN,NaN,NaN,NaN,NaN,exp_smoothing,,global


,metric,value,split,model,model_group_name,group
0,rmse,28.895677,train,exp_smoothing,,global
1,mae,24.030276,train,exp_smoothing,,global
2,mape,31.889076,train,exp_smoothing,,global
3,smape,26.904127,train,exp_smoothing,,global
4,r_squared,0.328295,train,exp_smoothing,,global
5,mda,65.853659,train,exp_smoothing,,global
6,rmse,27.601093,test,exp_smoothing,,global
7,mae,20.430424,test,exp_smoothing,,global
8,mape,16.022215,test,exp_smoothing,,global
9,smape,17.427549,test,exp_smoothing,,global


## 1.16 - Prophet Variations

### High Flexibility

In [50]:
from py_parsnip import prophet_reg

# Prophet with high flexibility
spec_prophet_flexible = prophet_reg(
    n_changepoints=50,
    changepoint_prior_scale=0.5,  # High flexibility
    seasonality_prior_scale=10.0
)

# Fit model
fit_prophet_flex = spec_prophet_flexible.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_prophet_flex = fit_prophet_flex.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs
outputs_prophet_flex, coefs_prophet_flex, stats_prophet_flex = fit_prophet_flex.extract_outputs()
display(outputs_prophet_flex)
display(coefs_prophet_flex)
display(stats_prophet_flex)

fig = plot_forecast(fit_prophet_flex, title="Sales Forecast")
fig.show()

Model fitted successfully!


,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,119.385717,137.65,18.264283,train,prophet_reg,,global
1,2020-05-01,113.53,115.188468,113.53,-1.658468,train,prophet_reg,,global
2,2020-06-01,43.31,60.711483,43.31,-17.401483,train,prophet_reg,,global
3,2020-07-01,79.77,65.729026,79.77,14.040974,train,prophet_reg,,global
4,2020-08-01,48.27,40.609995,48.27,7.660005,train,prophet_reg,,global
5,2020-09-01,98.01,108.903912,98.01,-10.893912,train,prophet_reg,,global
6,2020-10-01,49.77,69.893399,49.77,-20.123399,train,prophet_reg,,global
7,2020-11-01,35.48,37.218475,35.48,-1.738475,train,prophet_reg,,global
8,2020-12-01,63.98,89.797469,63.98,-25.817469,train,prophet_reg,,global
9,2021-01-01,109.76,116.288743,109.76,-6.528743,train,prophet_reg,,global


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,growth,linear,NaN,NaN,NaN,NaN,NaN,NaN,prophet_reg,,global
1,changepoint_prior_scale,0.5,NaN,NaN,NaN,NaN,NaN,NaN,prophet_reg,,global
2,seasonality_prior_scale,10.0,NaN,NaN,NaN,NaN,NaN,NaN,prophet_reg,,global
3,seasonality_mode,additive,NaN,NaN,NaN,NaN,NaN,NaN,prophet_reg,,global
4,n_changepoints,32,NaN,NaN,NaN,NaN,NaN,NaN,prophet_reg,,global


,metric,value,split,model,model_group_name,group
0,rmse,16.311094,train,prophet_reg,,global
1,mae,12.905772,train,prophet_reg,,global
2,mape,14.144755,train,prophet_reg,,global
3,smape,13.577319,train,prophet_reg,,global
4,r_squared,0.785968,train,prophet_reg,,global
5,mda,73.170732,train,prophet_reg,,global
6,rmse,70.25461,test,prophet_reg,,global
7,mae,64.115047,test,prophet_reg,,global
8,mape,51.49865,test,prophet_reg,,global
9,smape,73.316913,test,prophet_reg,,global


### Low Flexibility (Conservative)

In [51]:
# Prophet with low flexibility
spec_prophet_conservative = prophet_reg(
    n_changepoints=10,
    changepoint_prior_scale=0.01,  # Low flexibility
    seasonality_prior_scale=1.0
)

# Fit model
fit_prophet_cons = spec_prophet_conservative.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_prophet_cons = fit_prophet_cons.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs
outputs_prophet_cons, coefs_prophet_cons, stats_prophet_cons = fit_prophet_cons.extract_outputs()
display(outputs_prophet_cons)
display(coefs_prophet_cons)
display(stats_prophet_cons)

fig = plot_forecast(fit_prophet_cons, title="Sales Forecast")
fig.show()

Model fitted successfully!


,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,113.786519,137.65,23.863481,train,prophet_reg,,global
1,2020-05-01,113.53,113.648185,113.53,-0.118185,train,prophet_reg,,global
2,2020-06-01,43.31,70.209840,43.31,-26.899840,train,prophet_reg,,global
3,2020-07-01,79.77,77.822844,79.77,1.947156,train,prophet_reg,,global
4,2020-08-01,48.27,48.926817,48.27,-0.656817,train,prophet_reg,,global
5,2020-09-01,98.01,106.859031,98.01,-8.849031,train,prophet_reg,,global
6,2020-10-01,49.77,71.105010,49.77,-21.335010,train,prophet_reg,,global
7,2020-11-01,35.48,58.941615,35.48,-23.461615,train,prophet_reg,,global
8,2020-12-01,63.98,96.440040,63.98,-32.460040,train,prophet_reg,,global
9,2021-01-01,109.76,99.606593,109.76,10.153407,train,prophet_reg,,global


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,growth,linear,NaN,NaN,NaN,NaN,NaN,NaN,prophet_reg,,global
1,changepoint_prior_scale,0.01,NaN,NaN,NaN,NaN,NaN,NaN,prophet_reg,,global
2,seasonality_prior_scale,1.0,NaN,NaN,NaN,NaN,NaN,NaN,prophet_reg,,global
3,seasonality_mode,additive,NaN,NaN,NaN,NaN,NaN,NaN,prophet_reg,,global
4,n_changepoints,10,NaN,NaN,NaN,NaN,NaN,NaN,prophet_reg,,global


,metric,value,split,model,model_group_name,group
0,rmse,19.295564,train,prophet_reg,,global
1,mae,14.801271,train,prophet_reg,,global
2,mape,16.976513,train,prophet_reg,,global
3,smape,15.889741,train,prophet_reg,,global
4,r_squared,0.700479,train,prophet_reg,,global
5,mda,75.609756,train,prophet_reg,,global
6,rmse,71.972249,test,prophet_reg,,global
7,mae,63.825915,test,prophet_reg,,global
8,mape,50.966348,test,prophet_reg,,global
9,smape,76.531784,test,prophet_reg,,global


## 1.17 - Recursive Forecasting (Multi-Step Ahead)

In [52]:
from py_parsnip import recursive_reg, rand_forest

# Create specification - Recursive forecasting with Random Forest
spec_recursive = recursive_reg(
    base_model=rand_forest().set_mode('regression'),
    lags=[1, 2, 3, 6, 12],  # Use specific lags
    differentiation=None
)

# Fit model - requires date column as index
train_indexed = train_data.set_index('date')
fit_recursive = spec_recursive.fit(train_indexed, FORMULA_STR)

# Evaluate on test set
test_indexed = test_data.set_index('date')
fit_recursive = fit_recursive.evaluate(test_indexed)

print("Model fitted successfully!")

# Extract outputs
outputs_recursive, coefs_recursive, stats_recursive = fit_recursive.extract_outputs()
display(outputs_recursive)
display(coefs_recursive)
display(stats_recursive)

fig = plot_forecast(fit_recursive, title="Sales Forecast")
fig.show()

Model fitted successfully!


,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,NaN,137.65,NaN,train,recursive_reg,,global
1,2020-05-01,113.53,NaN,113.53,NaN,train,recursive_reg,,global
2,2020-06-01,43.31,NaN,43.31,NaN,train,recursive_reg,,global
3,2020-07-01,79.77,NaN,79.77,NaN,train,recursive_reg,,global
4,2020-08-01,48.27,NaN,48.27,NaN,train,recursive_reg,,global
5,2020-09-01,98.01,NaN,98.01,NaN,train,recursive_reg,,global
6,2020-10-01,49.77,NaN,49.77,NaN,train,recursive_reg,,global
7,2020-11-01,35.48,NaN,35.48,NaN,train,recursive_reg,,global
8,2020-12-01,63.98,NaN,63.98,NaN,train,recursive_reg,,global
9,2021-01-01,109.76,NaN,109.76,NaN,train,recursive_reg,,global


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,lag_1,0.040953,NaN,NaN,NaN,NaN,NaN,NaN,recursive_reg,,global
1,lag_2,0.182987,NaN,NaN,NaN,NaN,NaN,NaN,recursive_reg,,global
2,lag_3,0.020124,NaN,NaN,NaN,NaN,NaN,NaN,recursive_reg,,global
3,lag_6,0.075470,NaN,NaN,NaN,NaN,NaN,NaN,recursive_reg,,global
4,lag_12,0.025945,NaN,NaN,NaN,NaN,NaN,NaN,recursive_reg,,global


,metric,value,split,model,model_group_name,group
0,rmse,NaN,train,recursive_reg,,global
1,mae,NaN,train,recursive_reg,,global
2,mape,NaN,train,recursive_reg,,global
3,r_squared,NaN,train,recursive_reg,,global
4,rmse,26.142875,test,recursive_reg,,global
5,mae,18.435552,test,recursive_reg,,global
6,mape,13.076775,test,recursive_reg,,global
7,r_squared,-0.767765,test,recursive_reg,,global
8,formula,target ~ .,,recursive_reg,,global
9,n_obs_train,42,train,recursive_reg,,global


Recipe

In [53]:

# IMPUTATION (6 steps):
#   step_impute_median()   - Fill missing with median
#   step_impute_mean()     - Fill missing with mean
#   step_impute_mode()     - Fill missing with mode
#   step_impute_knn()      - KNN imputation
#   step_impute_bag()      - Bagged tree imputation
#   step_impute_linear()   - Linear model imputation

# NORMALIZATION (4 steps):
#   step_normalize()       - Z-score normalization
#   step_range()           - Min-max scaling [0,1]
#   step_center()          - Center to mean=0
#   step_scale()           - Scale to sd=1

# ENCODING (6 steps):
#   step_dummy()           - One-hot encoding
#   step_one_hot()         - One-hot (alternative)
#   step_target_encode()   - Target encoding
#   step_ordinal()         - Ordinal encoding
#   step_bin()             - Discretization
#   step_date()            - Extract date features

# FEATURE ENGINEERING (8 steps):
#   step_poly()            - Polynomial features
#   step_interact()        - Interaction terms
#   step_ns()              - Natural splines
#   step_bs()              - B-splines
#   step_pca()             - Principal components
#   step_log()             - Log transform
#   step_sqrt()            - Square root transform
#   step_inverse()         - Inverse transform

# FILTERING (6 steps):
#   step_corr()            - Remove correlated features
#   step_nzv()             - Remove zero-variance
#   step_filter_missing()  - Remove columns with too many NAs
#   step_select()          - Select specific columns
#   step_rm()              - Remove columns
#   step_filter()          - Row filtering

# ROW OPERATIONS (6 steps):
#   step_sample()          - Sample rows
#   step_filter()          - Filter rows by condition
#   step_slice()           - Select row ranges
#   step_arrange()         - Sort rows
#   step_shuffle()         - Shuffle rows
#   step_lag()             - Create lagged features

# TRANSFORMATIONS (6 steps):
#   step_mutate()          - Create/modify columns
#   step_discretize()      - Discretization
#   step_cut()             - Cut into bins
#   step_BoxCox()          - Box-Cox transform
#   step_YeoJohnson()      - Yeo-Johnson transform
#   step_other()           - Pool infrequent categories

# TIME SERIES (4 steps):
#   step_lag()             - Create lags
#   step_diff()            - Differencing
#   step_date()            - Extract date features
#   step_timeseries_signature() - Comprehensive time features


In [54]:
'Create a simple recipe'

rec = (
    recipe()
    .step_normalize()  # Normalize numeric columns
    
)
# Fit on training data
rec_fit = rec.prep(train_data)

# Apply to training data
train_processed = rec_fit.bake(train_data)

print("\nProcessed training data:")

# Apply to test data (using training parameters!)
test_processed = rec_fit.bake(test_data)
print("\nProcessed test data:")
display(train_processed)
display(test_processed)



Processed training data:

Processed test data:


,date,mean_med_diesel_crack_input1_trade_month_lag2,mean_nwe_hsfo_crack_trade_month_lag1,mean_nwe_lsfo_crack_trade_month,mean_nwe_ulsfo_crack_trade_month_lag3,mean_sing_gasoline_vs_vlsfo_trade_month,mean_sing_vlsfo_crack_trade_month_lag3,new_sweet_sr_margin,target,totaltar
0,2020-04-01,-0.088770,-0.094595,0.046119,1.487556,-2.378036,2.031849,0.358746,1.096099,-0.237232
1,2020-05-01,0.172544,0.089438,1.195442,0.793182,-0.956278,1.074411,0.389075,0.411977,-0.237232
2,2020-06-01,0.402280,0.137482,-0.014980,-0.283450,-0.090439,-0.246999,2.220933,-1.579693,-0.237232
3,2020-07-01,0.427807,0.277542,0.105567,-0.597906,0.118757,-0.654823,-0.253895,-0.545568,-0.237232
4,2020-08-01,0.239857,0.358972,0.118778,-1.317998,-0.297698,-0.986637,-0.602676,-1.439011,-0.237232
5,2020-09-01,0.183889,0.378516,0.209601,-0.954445,0.107135,-1.002717,-0.420703,-0.028221,-0.237232
6,2020-10-01,0.141723,0.420859,0.579498,-0.907686,-0.096250,-1.053877,-0.796780,-1.396466,-0.237232
7,2020-11-01,0.161577,0.393987,0.850315,-0.934573,-0.694783,-0.879931,-0.551117,-1.801778,-0.237232
8,2020-12-01,0.187859,0.560105,0.708301,-1.072512,-0.743208,-0.992484,0.216201,-0.993424,-0.237232
9,2021-01-01,0.194855,0.345129,0.657110,-0.839885,-0.747082,-0.700137,0.201036,0.305047,-0.237232


,date,mean_med_diesel_crack_input1_trade_month_lag2,mean_nwe_hsfo_crack_trade_month_lag1,mean_nwe_lsfo_crack_trade_month,mean_nwe_ulsfo_crack_trade_month_lag3,mean_sing_gasoline_vs_vlsfo_trade_month,mean_sing_vlsfo_crack_trade_month_lag3,new_sweet_sr_margin,target,totaltar
0,2023-10-01,-0.603078,0.495775,-0.521937,-0.600244,-0.260895,-0.252846,1.486976,0.245201,-0.237232
1,2023-11-01,-0.680981,0.024293,-0.589642,-0.463473,-0.853617,-0.238228,0.813678,0.549255,-0.237232
2,2023-12-01,-0.685519,-0.290842,-0.553312,-0.851575,1.366184,-0.413637,-1.078838,0.397795,-0.237232
3,2024-01-01,-0.589653,-0.052251,-0.350199,-0.721818,0.426739,-0.227996,0.234398,0.670650,-0.237232
4,2024-02-01,-0.479418,1.820648,-0.738261,-0.322026,0.878060,0.545263,1.080571,0.853309,-0.237232
5,2024-03-01,-0.467694,-0.179283,-0.578082,-0.099920,0.568140,0.080430,0.725724,1.320736,0.432258
6,2024-04-01,-0.509860,1.814133,-0.802663,-0.315013,1.211224,-0.156371,0.774250,1.679816,0.983528
7,2024-05-01,-0.574149,-0.285142,-0.829084,0.344293,0.637872,0.179828,0.574080,0.142809,0.050824
8,2024-06-01,-0.677388,-0.042480,-0.814222,0.216874,0.064521,0.138900,-0.763419,2.105265,0.983528
9,2024-07-01,-0.604402,0.142367,-0.612760,0.034513,-0.028455,-0.327394,-0.017331,0.771340,1.244599


## 2.1 - Workflows

In [55]:
# FORMULA APPROACH

# Simple formula-based workflow
# Limitations: Can't easily normalize, encode categoricals, etc.
wf_formula = (
    workflow()
    .add_formula(FORMULA_STR)
    .add_model(linear_reg())
)

fit_formula = wf_formula.fit(train_processed)
fit_formula = fit_formula.evaluate(test_processed)

outputs_formula, coefs_formula, stats_formula = fit_formula.extract_outputs()
display(outputs_formula)
display(coefs_formula)
display(stats_formula)


,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,1.096099,0.039546,1.096099,1.056553,train,linear_reg,,global
1,2020-05-01,0.411977,-0.016712,0.411977,0.428689,train,linear_reg,,global
2,2020-06-01,-1.579693,0.094006,-1.579693,-1.673700,train,linear_reg,,global
3,2020-07-01,-0.545568,-0.097141,-0.545568,-0.448427,train,linear_reg,,global
4,2020-08-01,-1.439011,-0.281262,-1.439011,-1.157749,train,linear_reg,,global
5,2020-09-01,-0.028221,-0.178389,-0.028221,0.150168,train,linear_reg,,global
6,2020-10-01,-1.396466,-0.282384,-1.396466,-1.114082,train,linear_reg,,global
7,2020-11-01,-1.801778,-0.393669,-1.801778,-1.408109,train,linear_reg,,global
8,2020-12-01,-0.993424,-0.395386,-0.993424,-0.598038,train,linear_reg,,global
9,2021-01-01,0.305047,-0.312078,0.305047,0.617125,train,linear_reg,,global


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,Intercept,2.628306e-16,0.151184,1.738479e-15,1.000000,-0.307587,0.307587,NaN,linear_reg,,global
1,mean_med_diesel_crack_input1_trade_month_lag2,-9.410508e-02,0.168384,-5.588728e-01,0.580022,-0.436684,0.248474,1.240473,linear_reg,,global
2,mean_nwe_hsfo_crack_trade_month_lag1,-1.121007e-01,0.172873,-6.484566e-01,0.521177,-0.463814,0.239612,1.307501,linear_reg,,global
3,mean_nwe_lsfo_crack_trade_month,-1.038090e-01,0.198654,-5.225630e-01,0.604769,-0.507973,0.300355,1.726553,linear_reg,,global
4,mean_nwe_ulsfo_crack_trade_month_lag3,5.714816e-02,0.655382,8.719818e-02,0.931041,-1.276238,1.390534,18.792161,linear_reg,,global
5,mean_sing_gasoline_vs_vlsfo_trade_month,2.015905e-01,0.192450,1.047494e+00,0.302484,-0.189953,0.593133,1.620407,linear_reg,,global
6,mean_sing_vlsfo_crack_trade_month_lag3,1.464219e-01,0.653304,2.241250e-01,0.824042,-1.182736,1.475580,18.673177,linear_reg,,global
7,new_sweet_sr_margin,5.051546e-02,0.213297,2.368318e-01,0.814251,-0.383440,0.484471,1.990470,linear_reg,,global
8,totaltar,-4.389148e-01,0.250863,-1.749618e+00,0.089480,-0.949300,0.071470,2.753345,linear_reg,,global


,metric,value,split,model,model_group_name,group
0,rmse,0.868487,train,linear_reg,,global
1,mae,0.708766,train,linear_reg,,global
2,mape,267.502038,train,linear_reg,,global
3,smape,150.817641,train,linear_reg,,global
4,r_squared,0.24573,train,linear_reg,,global
5,mda,58.536585,train,linear_reg,,global
6,adj_r_squared,0.062877,train,linear_reg,,global
7,rmse,1.154199,test,linear_reg,,global
8,mae,0.943515,test,linear_reg,,global
9,mape,211.503325,test,linear_reg,,global


In [56]:
# from py_recipes.selectors import (
#     all_numeric,      # All numeric columns
#     all_nominal,      # All categorical columns
#     all_predictors,   # All predictor columns
#     all_outcomes,     # All outcome columns
#     has_role,         # Columns with specific role
#     has_type,         # Columns with specific type
# )

print("APPROACH 2: Recipe-Based Workflow (Powerful)")

# Recipe-based workflow with advanced feature engineering
# Advantages: Normalization, encoding, imputation, feature creation, etc.

# For recipes, we prep/bake outside the workflow, then use formula in workflow
rec = (
    recipe()  # Create empty recipe
    .step_normalize()  # Normalize numeric features (z-score) - None = all numeric
)

rec = (
    recipe()
    .step_impute_median()      # 1. Impute
    #.step_boxcox(contains("crack"))       # 2. Transform
    .step_boxcox()       # 2. Transform
    #.step_normalize()          # 3. Normalize
    #.step_range()          # 3. Normalize
    #.step_center()          # 3. Normalize
    #.step_scale()          # 3. Normalize
    #.step_dummy(["category"])  # 4. Encode
    #.step_corr(threshold=0.9)  # 5. Filter
    #.step_poly(['totaltar'], degree=2)
    #.step_interact([('totaltar', 'mean_med_diesel_crack_input1_trade_month_lag2')])
    #.step_ns('mean_med_diesel_crack_input1_trade_month_lag2', df = 2)
    #.step_pca(["totaltar", "v", "mean_nwe_hsfo_crack_trade_month_lag1"], num_comp=3)
    #.step_nzv()
    #.step_filter_missing(threshold=0.5) # Remove columns with >50% missing
    #.step_lag("mean_med_diesel_crack_input1_trade_month_lag2", lags=[1, 7, 14, 28])  # Lagged features
    #.step_diff(["mean_med_diesel_crack_input1_trade_month_lag2"], lag=1)                # First difference
    #.step_timeseries_signature("date")        # Extract time features
    .step_select_corr(outcome='target', threshold=0.9, method='multicollinearity')  # Remove highly correlated
)

# Prep the recipe on training data
rec_prepped = rec.prep(train_data)

# Apply to both train and test
train_processed_recipe = rec_prepped.bake(train_data)
test_processed_recipe = rec_prepped.bake(test_data)

print(train_processed_recipe.columns)

wf_recipe = (
    workflow()
    # Formula on preprocessed data
    #.add_formula(FORMULA_STR)
    .add_recipe(rec)
    .add_model(linear_reg())
)

fit_recipe = wf_recipe.fit(train_data)
fit_recipe = fit_recipe.evaluate(test_data)

outputs_recipe, coefs_recipe, stats_recipe = fit_recipe.extract_outputs()
display(outputs_recipe)
display(coefs_recipe)
display(stats_recipe)

APPROACH 2: Recipe-Based Workflow (Powerful)
Index(['date', 'mean_med_diesel_crack_input1_trade_month_lag2',
       'mean_nwe_hsfo_crack_trade_month_lag1',
       'mean_nwe_lsfo_crack_trade_month',
       'mean_nwe_ulsfo_crack_trade_month_lag3',
       'mean_sing_gasoline_vs_vlsfo_trade_month', 'new_sweet_sr_margin',
       'target', 'totaltar'],
      dtype='object')


,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,80.440353,61.045674,80.440353,19.394679,train,linear_reg,,global
1,2020-05-01,67.926466,61.655635,67.926466,6.270830,train,linear_reg,,global
2,2020-06-01,28.882702,61.934255,28.882702,-33.051552,train,linear_reg,,global
3,2020-07-01,49.766772,57.931067,49.766772,-8.164295,train,linear_reg,,global
4,2020-08-01,31.833518,45.094697,31.833518,-13.261179,train,linear_reg,,global
5,2020-09-01,59.685178,53.759895,59.685178,5.925284,train,linear_reg,,global
6,2020-10-01,32.717617,52.878267,32.717617,-20.160651,train,linear_reg,,global
7,2020-11-01,24.127453,50.145055,24.127453,-26.017602,train,linear_reg,,global
8,2020-12-01,40.928859,48.264781,40.928859,-7.335922,train,linear_reg,,global
9,2021-01-01,65.939461,52.026295,65.939461,13.913166,train,linear_reg,,global


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,Intercept,19.124256,27.406301,0.697805,0.490044,-36.572048,74.820560,NaN,linear_reg,,global
1,mean_med_diesel_crack_input1_trade_month_lag2,-0.032158,0.057680,-0.557526,0.580821,-0.149379,0.085062,1.222203,linear_reg,,global
2,mean_nwe_hsfo_crack_trade_month_lag1,-0.138910,0.241495,-0.575210,0.568937,-0.629688,0.351867,1.155154,linear_reg,,global
3,mean_nwe_lsfo_crack_trade_month,-0.233839,0.542270,-0.431222,0.669029,-1.335863,0.868186,1.416318,linear_reg,,global
4,mean_nwe_ulsfo_crack_trade_month_lag3,22.070911,14.922514,1.479034,0.148337,-8.255287,52.397109,1.591220,linear_reg,,global
5,mean_sing_gasoline_vs_vlsfo_trade_month,0.702057,0.631326,1.112035,0.273925,-0.580952,1.985066,1.395226,linear_reg,,global
6,new_sweet_sr_margin,0.215382,1.130854,0.190460,0.850081,-2.082790,2.513555,1.825994,linear_reg,,global
7,totaltar,-0.183634,0.082476,-2.226529,0.032709,-0.351245,-0.016024,1.947766,linear_reg,,global


,metric,value,split,model,model_group_name,group
0,rmse,16.089461,train,linear_reg,,global
1,mae,13.080208,train,linear_reg,,global
2,mape,25.354143,train,linear_reg,,global
3,smape,22.677264,train,linear_reg,,global
4,r_squared,0.286434,train,linear_reg,,global
5,mda,58.536585,train,linear_reg,,global
6,adj_r_squared,0.139523,train,linear_reg,,global
7,rmse,21.658552,test,linear_reg,,global
8,mae,18.53875,test,linear_reg,,global
9,mape,23.987486,test,linear_reg,,global


In [57]:
print("\n" + "="*70)
print("APPROACH 3: Recipe Without Workflow (Manual prep/bake)")
print("="*70)

# You can also use recipes without workflows - just prep/bake directly
rec_manual = (
    recipe()
    .step_normalize()
)

# Prep on training data
rec_manual_prepped = rec_manual.prep(train_data)

# Bake both datasets
train_manual = rec_manual_prepped.bake(train_data)
test_manual = rec_manual_prepped.bake(test_data)

spec_manual = linear_reg()
fit_manual = spec_manual.fit(train_manual, "target ~ . -date")
fit_manual = fit_manual.evaluate(test_manual)

outputs_manual, coefs_manual, stats_manual = fit_manual.extract_outputs()
display(outputs_manual)
display(coefs_manual)
display(stats_manual)



APPROACH 3: Recipe Without Workflow (Manual prep/bake)


,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,1.096099,0.039546,1.096099,1.056553,train,linear_reg,,global
1,2020-05-01,0.411977,-0.016712,0.411977,0.428689,train,linear_reg,,global
2,2020-06-01,-1.579693,0.094006,-1.579693,-1.673700,train,linear_reg,,global
3,2020-07-01,-0.545568,-0.097141,-0.545568,-0.448427,train,linear_reg,,global
4,2020-08-01,-1.439011,-0.281262,-1.439011,-1.157749,train,linear_reg,,global
5,2020-09-01,-0.028221,-0.178389,-0.028221,0.150168,train,linear_reg,,global
6,2020-10-01,-1.396466,-0.282384,-1.396466,-1.114082,train,linear_reg,,global
7,2020-11-01,-1.801778,-0.393669,-1.801778,-1.408109,train,linear_reg,,global
8,2020-12-01,-0.993424,-0.395386,-0.993424,-0.598038,train,linear_reg,,global
9,2021-01-01,0.305047,-0.312078,0.305047,0.617125,train,linear_reg,,global


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,Intercept,2.628306e-16,0.151184,1.738479e-15,1.000000,-0.307587,0.307587,NaN,linear_reg,,global
1,mean_med_diesel_crack_input1_trade_month_lag2,-9.410508e-02,0.168384,-5.588728e-01,0.580022,-0.436684,0.248474,1.240473,linear_reg,,global
2,mean_nwe_hsfo_crack_trade_month_lag1,-1.121007e-01,0.172873,-6.484566e-01,0.521177,-0.463814,0.239612,1.307501,linear_reg,,global
3,mean_nwe_lsfo_crack_trade_month,-1.038090e-01,0.198654,-5.225630e-01,0.604769,-0.507973,0.300355,1.726553,linear_reg,,global
4,mean_nwe_ulsfo_crack_trade_month_lag3,5.714816e-02,0.655382,8.719818e-02,0.931041,-1.276238,1.390534,18.792161,linear_reg,,global
5,mean_sing_gasoline_vs_vlsfo_trade_month,2.015905e-01,0.192450,1.047494e+00,0.302484,-0.189953,0.593133,1.620407,linear_reg,,global
6,mean_sing_vlsfo_crack_trade_month_lag3,1.464219e-01,0.653304,2.241250e-01,0.824042,-1.182736,1.475580,18.673177,linear_reg,,global
7,new_sweet_sr_margin,5.051546e-02,0.213297,2.368318e-01,0.814251,-0.383440,0.484471,1.990470,linear_reg,,global
8,totaltar,-4.389148e-01,0.250863,-1.749618e+00,0.089480,-0.949300,0.071470,2.753345,linear_reg,,global


,metric,value,split,model,model_group_name,group
0,rmse,0.868487,train,linear_reg,,global
1,mae,0.708766,train,linear_reg,,global
2,mape,267.502038,train,linear_reg,,global
3,smape,150.817641,train,linear_reg,,global
4,r_squared,0.24573,train,linear_reg,,global
5,mda,58.536585,train,linear_reg,,global
6,adj_r_squared,0.062877,train,linear_reg,,global
7,rmse,1.154199,test,linear_reg,,global
8,mae,0.943515,test,linear_reg,,global
9,mape,211.503325,test,linear_reg,,global


In [58]:
# method chaining
# Full pipeline in one chain
outputs2, coefficients2, stats2 = (
    workflow()
    .add_formula(FORMULA_STR)
    .add_model(linear_reg().set_engine("sklearn"))
    .fit(train_data)
    .evaluate(test_data)
    .extract_outputs()
)

print("Method chaining complete!")
print(f"\nTest R²: {stats2[(stats2['metric'] == 'r_squared') & (stats2['split'] == 'test')]['value'].values[0]:.4f}")

Method chaining complete!

Test R²: -3.2832


In [59]:
# # Extract components
# model_fit = wf_recipe.extract_fit_parsnip()
# preprocessor = wf_recipe.extract_preprocessor()
# model_spec = wf_recipe.extract_spec_parsnip()

# print("Extracted components:")
# print(f"  Model fit type: {type(model_fit)}")
# print(f"  Preprocessor: {preprocessor}")
# print(f"  Model spec: {model_spec.mode} with {model_spec.engine} engine")

# # Access underlying sklearn model
# sklearn_model = model_fit.fit_data["model"]
# print(f"\n  Underlying model type: {type(sklearn_model)}")
# print(f"  Number of coefficients: {len(sklearn_model.coef_)}")

In [60]:
# Create recipe
rec_for_model = (
    recipe()
    .step_impute_mean()
    .step_normalize()
    #.step_remove("date")
)

# Create workflow
wf = (
    workflow()
    .add_recipe(rec_for_model)
    .add_model(linear_reg().set_engine("sklearn"))
)

print("Workflow created with recipe preprocessing")
print(f"Preprocessor: {type(wf.preprocessor).__name__}")
print(f"Model: {wf.spec.model_type}")

# Fit workflow (recipe is automatically prepped)
wf_fit = wf.fit(train_data)

print("Workflow fitted successfully")
print(f"Preprocessor type: {type(wf_fit.pre).__name__}")

Workflow created with recipe preprocessing
Preprocessor: Recipe
Model: linear_reg
Workflow fitted successfully
Preprocessor type: PreparedRecipe


In [61]:
# Define different formulas (preprocessing strategies)
formulas = [
    "target ~ mean_med_diesel_crack_input1_trade_month_lag2 + mean_nwe_hsfo_crack_trade_month_lag1",                                                        # Minimal model
    "target ~ mean_med_diesel_crack_input1_trade_month_lag2 + mean_nwe_hsfo_crack_trade_month_lag1 + mean_nwe_lsfo_crack_trade_month",                      # Add more features
    "target ~ mean_med_diesel_crack_input1_trade_month_lag2 + mean_nwe_hsfo_crack_trade_month_lag1 + mean_nwe_lsfo_crack_trade_month + totaltar",           # All features
    "target ~ mean_med_diesel_crack_input1_trade_month_lag2 + mean_nwe_hsfo_crack_trade_month_lag1 + mean_nwe_lsfo_crack_trade_month + I(mean_med_diesel_crack_input1_trade_month_lag2*mean_nwe_hsfo_crack_trade_month_lag1)",           # With interaction
    "target ~ mean_med_diesel_crack_input1_trade_month_lag2 + mean_nwe_hsfo_crack_trade_month_lag1 + mean_nwe_lsfo_crack_trade_month + I(mean_med_diesel_crack_input1_trade_month_lag2**2) + I(mean_nwe_hsfo_crack_trade_month_lag1**2)" # With polynomials
]

# Descriptive IDs for each formula
formula_ids = [
    "minimal",
    "medium",
    "full",
    "interaction",
    "polynomial"
]

print(f"Number of preprocessing strategies: {len(formulas)}")

Number of preprocessing strategies: 5
